### Transformer Protein Scaffold Filling - p5A

In [20]:
import os
os.system('color')
import random
from termcolor import colored
from typing import List, Optional
from copy import deepcopy
import csv
from enum import Enum

import tensorflow as tf
from keras import layers, Model
import numpy as np
from sklearn.preprocessing import OneHotEncoder
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator

import utility as ut
import pickle as pk
os.system('color')
random.seed(0)
np.random.seed(0)

DATAPATH = os.path.join(os.getcwd(), "newdatasets_p5A")
RESULTS_PATH = os.path.join(os.getcwd(), "transformer_results-p5A")
from keras.utils.vis_utils import plot_model

sh: 1: color: not found
sh: 1: color: not found


In [2]:
#!pip install reportlab

In [21]:
'''
Trial summary
Hyperparameters:
conv1_filter_size: 10
conv2_filter_size: 9
conv1_filters: 50
conv2_filters: 66
bridge_filters: 120
max_pool: 2
dropout_: : 0.35
Score: 0.9727120995521545


Hyperparameters:
conv1_filter_size: 5
conv2_filter_size: 9
conv1_filters: 46
conv2_filters: 90
bridge_filters: 160
bridge_filter_size: 5
max_pool: 2
dropout_: : 0.5
Score: 0.9742878675460815
'''

# # =============================================================================
# class Encoder(layers.Layer):
#     """Encoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=10, conv2_filter_size=9, maxpool=2, dropout=0.25, name="encoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1D(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1D(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.maxpool = layers.MaxPooling1D(maxpool, padding="same")
#         self.dropout = layers.Dropout(dropout)
        

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.maxpool(self.conv2(self.dropout(self.maxpool(self.conv1(inputs))))))

# # =============================================================================
# class Decoder(layers.Layer):
#     """Decoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=5, conv2_filter_size=9, maxpool=2, dropout=0.25, name="decoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1DTranspose(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1DTranspose(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.upsample = layers.UpSampling1D(maxpool)
#         self.dropout = layers.Dropout(dropout)

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.upsample(self.conv2(self.dropout(self.upsample(self.conv1(inputs))))))

# # =============================================================================
# class Autoencoder(Model):
#     """Autoencoder"""
#     # -------------------------------------------------------------------------
#     def __init__(self, num_classes, name="autoencoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.num_classes = num_classes
#         self.hyperparameters = {
#             "conv1_filters": 46,
#             "conv2_filters": 90,
#             "conv1_filter_size": 5,
#             "conv2_filter_size": 9,
#             "bridge_filters": 160,
#             "bridge_filter_size": 5,
#             "dropout": 0.50,
#             "maxpool": 5,
#         }

#         hp = self.hyperparameters

#         self.encoder = Encoder(hp["conv1_filters"], hp["conv2_filters"])
#         self.bridge = layers.Conv1D(hp["bridge_filters"], hp["bridge_filter_size"], padding="same", activation="relu")
#         self.decoder = Decoder(hp["conv2_filters"], hp["conv1_filters"])
#         self.finallayer = layers.Conv1D(self.num_classes, hp["conv1_filter_size"], padding="same", activation="softmax")

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.finallayer(self.decoder(self.bridge(self.encoder(inputs))))

    
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class Transformer(Model):
    def __init__(self, input_shape_, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, 
                 mlp_dropout=0,name="transformers", **kwargs):
        super().__init__(name=name, **kwargs)
        self.input_shape_ = input_shape_
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout

    def transformer_encoder(self, inputs):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
        )(x, x)
        x = layers.Dropout(self.dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(self.dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    def build_model(self):
        print("Input shape:", self.input_shape_)
        inputs = tf.keras.Input(shape=self.input_shape_)
        x = inputs
        for _ in range(self.num_transformer_blocks):
            x = self.transformer_encoder(x)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in self.mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(self.mlp_dropout)(x)
        outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
        outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
        return models.Model(inputs, outputs)

#     def call(self, inputs):
#         x = inputs
#         for _ in range(self.num_transformer_blocks):
#             x = self.transformer_encoder(x)

#         x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
#         for dim in self.mlp_units:
#             x = layers.Dense(dim, activation="relu")(x)
#             x = layers.Dropout(self.mlp_dropout)(x)
#         outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
#         outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
#         return outputs
        
# =============================================================================
class PaddingType(Enum):
    ZERO=1
    TRUNCATE=2
    EMPTY=3

# =============================================================================

class NoisificationMethod(Enum):
    RANDOMSCATTER=1
    RANDOMCONTIG=2

weights = None

# =============================================================================
class ProteinScaffoldFixer():
    """Class to correct errors in a protein scaffold and fill gaps"""

    # -------------------------------------------------------------------------
    def __init__(self, 
                output_seqs, 
                paddingtype=PaddingType.EMPTY, 
                noise_percent=np.array([0.20, 0.30, 0.40]),#0.2, 
                noisemethod=NoisificationMethod.RANDOMSCATTER, 
                numgaps=np.array([4, 6, 8, 10]),#5, 
                mingapsize=3, 
                mincontigsize=1, 
                epochs=250,
                optimizer="adam",
                early_stopping=False):

        self.epochs = epochs
        self.numgaps = numgaps#added today
        self.optimizer = optimizer
        self.paddingtype = paddingtype
        self.noise_percent = noise_percent
        self.early_stopping = early_stopping

        self.classes = np.array(["-", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z'])
        self.ohe = OneHotEncoder(sparse_output=False, categories=[self.classes])

        self.max_seq_length = max((len(seq) for seq in output_seqs))
        self.maxpool = 2
        while self.max_seq_length % (self.maxpool * self.maxpool) != 0:
            self.max_seq_length += 1

        self.output_seqs = output_seqs
        #self.input_seqs = self.noisify_sequences(output_seqs, noise_percent, noisemethod, numgaps, mingapsize, mincontigsize)
        #########################
        self.output_seqs_all = []
        self.input_seqs_all = []
        
        for n_gaps in numgaps:
            for p_missing in noise_percent:
                self.output_seqs_temp = output_seqs
                self.input_seqs_temp = self.noisify_sequences(self.output_seqs_temp, p_missing, noisemethod, n_gaps, mingapsize=3, mincontigsize=1)
                self.output_seqs_all += self.output_seqs_temp
                self.input_seqs_all += self.input_seqs_temp
        #########################
        self.train_x = self.preprocess_sequences(self.input_seqs_all)
        self.train_y = self.preprocess_sequences(self.output_seqs_all)

        self.history = None

        #self.autoencoder = Autoencoder(len(self.classes))
        ### my additions ######
        self.input_shape = self.train_x.shape[1:]#(440,24)
        self.head_size=256
        self.num_heads=4
        self.ff_dim=4
        self.num_transformer_blocks=4#3
        self.mlp_units=[512,128]
        self.mlp_dropout=0.4
        self.dropout=0.25
        self.transformer = Transformer(self.input_shape, self.head_size, self.num_heads, self.ff_dim, 
                                self.num_transformer_blocks, self.mlp_units, self.dropout, self.mlp_dropout)
        #self.transformer = self.transformer.build_model()
    # =============================================================================
    def noisify_sequences(self, seqs, noise_percent, noisemethod, numgaps=5, mingapsize=3, mincontigsize=3):
        if noisemethod == NoisificationMethod.RANDOMSCATTER:
            return self.noisify_by_random_scatter(seqs, noise_percent)
        elif noisemethod == NoisificationMethod.RANDOMCONTIG:
            return self.noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize)

    # =============================================================================
    def noisify_by_random_scatter(self, seqs, noise_percent):
        sequences = deepcopy(seqs)
        # To noisify our input data, we will replace random amino acids with something else
        for seq in sequences:

            # We randomly sample from all the possible indices of seq
            indices_to_replace = random.sample(range(len(seq)), int(noise_percent * len(seq)))
            seq[indices_to_replace] = "-"

        return sequences

    # =============================================================================
    def noisify_by_random_contigs(self, seqs, noise_percent, numgaps, mingapsize, mincontigsize):

        sequences = deepcopy(seqs)
        for seqind, seq in enumerate(sequences):

            amino_acid_length = len(seq)
            amino_acids_to_replace = int(amino_acid_length * noise_percent)

            # The idea is to build a gap queue, randomly putting each amino acid to replace into each gap
            # Then, we build a contig queue, randomly putting each amino acid into each contig
            gap_queue = np.zeros(numgaps).astype(int)
            for _ in range(amino_acids_to_replace):

                # Any gap can be considered
                valid_gaps = np.arange(numgaps)

                # ... so long as we have no underfilled gaps
                underfilled = gap_queue < mingapsize
                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_gaps = valid_gaps[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                gap_queue[np.random.choice(valid_gaps)] += 1

            # There can always be one more contig than gaps (if there's a contig at beginnning and end of sequence)
            contig_queue = np.zeros(numgaps+1).astype(int)

            # We have to allocate all the amino acids NOT in gaps into contigs BETWEEN the gaps
            for _ in range(amino_acid_length - amino_acids_to_replace):

                # Any contig can be considered
                valid_contigs = np.arange(len(contig_queue))

                # ... so long as we have no underfilled contigs
                underfilled = contig_queue < mincontigsize

                # The exceptions are the first and last contigs. They are never considered underfilled
                underfilled[0] = False
                underfilled[-1] = False

                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_contigs = valid_contigs[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                contig_queue[np.random.choice(valid_contigs)] += 1
            
            # Once we have determined gap_queue and contig_queue, we iterate over them to set the gaps equal
            # to our blank amino acid character

            sequence_pointer = 0
            iscontig=True
            while sequence_pointer < len(seq):
                if iscontig:
                    # Don't do anything for the contig except increment the pointer and pop off the contig queue
                    sequence_pointer += contig_queue[0]
                    contig_queue = np.delete(contig_queue, 0)
                else:
                    seq[sequence_pointer:sequence_pointer+gap_queue[0]] = '-'
                    sequence_pointer += gap_queue[0]
                    gap_queue = np.delete(gap_queue, 0)

                # We alternate between contigs and gaps
                iscontig = not iscontig

        return sequences

    # -------------------------------------------------------------------------
    def predict_sequence(self, seq, predict_only_gaps):

        scaffold = self.preprocess_sequences([seq])
        #pred = self.autoencoder.predict(scaffold).reshape(self.max_seq_length, len(self.classes))
        pred = self.transformer.predict(scaffold).reshape(self.max_seq_length, len(self.classes))

        # Set the probability of empty "-" to zero, since we always want to predict something
        emptyclass = np.where(self.ohe.transform(np.array("-").reshape(-1, 1))[0])[0][0]
        pred[:, emptyclass] = 0.0

        # Convert the probability distribution to a one-hot encoded vector
        mask = pred == np.amax(pred, axis=1).reshape(pred.shape[0], 1)
        indices = list((i, np.where(mask[i])[0][0]) for i in range(mask.shape[0]))

        pred = np.zeros(pred.shape)
        for i in indices:
            pred[i] = 1

        # Then we can use our one hot encoder to convert back to the original sequence of classes
        pred = self.ohe.inverse_transform(pred[:len(seq), :len(self.classes)]).reshape(len(seq))

        if predict_only_gaps:
            # We only care about predicting the gaps in seq, so replace amino acids in prediction with original nongaps
            nongaps = np.where(seq != '-')[0]
            pred[nongaps] = seq[nongaps]

        return pred

    # =============================================================================
    def preprocess_sequences(self, seqs: List[np.array]) -> np.array:

        # the value -1 lets numpy know to infer the shape. So it's just a column vector of length num_samples
        seqs = [np.array(seq).reshape(-1, 1) for seq in seqs]

        # One-hot encode each sequence
        seqs = [self.ohe.fit_transform(seq) for seq in seqs]

        if self.paddingtype == PaddingType.ZERO:
            # The sequences may have different lengths, so we will pad them with zeros
            # We are padding to fill up to max length, then we can turn it into a single numpy tensor
            return np.array([np.pad(seq, ((0, self.max_seq_length-len(seq)), (0, 0))) for seq in seqs])

        elif self.paddingtype == PaddingType.EMPTY:

            # We pad with the "empty" class
            emptyclass = self.ohe.fit_transform(np.array("-").reshape(-1, 1))[0]

            # dynamically extend each seq by enough emptyvals to make a single sequence length
            return np.array([np.vstack((seq, *(emptyclass for _ in range(self.max_seq_length-len(seq))))) for seq in seqs])

        elif self.paddingtype == PaddingType.TRUNCATE:
            # FIXME: Implement me
            raise Exception("Have not implemented Padding Type TRUNCATE!")

    # =============================================================================
    def train(self, verbose="auto", optimizer="adam", epochs=None):
        self.optimizer = optimizer
        if epochs:
            self.epochs = epochs

#         self.autoencoder.compile(optimizer=self.optimizer, loss="categorical_crossentropy",
#                                  metrics=["accuracy"])
        self.transformer = self.transformer.build_model()
        self.transformer.compile(loss="categorical_crossentropy",
                                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                                    metrics=["accuracy"],)
        callbacks = []
        if self.early_stopping:
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10))
        #self.history = self.autoencoder.fit(self.train_x, self.train_y, epochs=self.epochs, validation_split=0.15, verbose=verbose, callbacks=callbacks)
        self.history = self.transformer.fit(self.train_x, self.train_y, epochs=self.epochs, batch_size=64,
                                            validation_split=0.15, verbose=verbose, callbacks=callbacks)
        # Save the history to a file using pickle
        with open('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-p5A/model_p5A/model_history.pkl', 'wb') as file:
            pk.dump(self.history, file)
        #plot_model(self.transformer, to_file='transformer_plot.png', show_shapes=True, show_layer_names=True)
        #print(self.transformer.summary())
        self.transformer.save('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-p5A/model_p5A/transormer_model_p5A.h5')
        
# =============================================================================
def highlight_indices(seq: np.array, indices: np.array, color: str):
    # We use deepcopy to prevent mutation and we cast to object so that we can treat contents as python strings
    # otherwise, it gets messed up as it treats each element as a single character
    newseq = deepcopy(seq).astype('object')
    newseq[indices] = np.vectorize(lambda x: colored(x, color, attrs=["bold"]))(seq[indices])
    return newseq

# =============================================================================
def print_sequence(seq, 
                   header: str=None, 
                   incorrect_indices: Optional[np.array]=None, 
                   correct_indices: Optional[np.array]=None):

    newseq = deepcopy(seq)
    if correct_indices is not None and correct_indices.size != 0:
        newseq = highlight_indices(newseq, correct_indices, "green")
    if incorrect_indices is not None and incorrect_indices.size != 0:
        newseq = highlight_indices(newseq, incorrect_indices, "red")

    line_length = 40
    if header:
        print(header)
    print("=" * line_length)

    i = 0
    while i < len(newseq):
        print(" ".join(newseq[i: i+line_length]))
        i += line_length

# =============================================================================
def get_sequences(fasta_file: str) -> List[np.array]:
    sequences = []
    lines = []
    with open(fasta_file, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append(np.array([c for c in "".join(parts)]))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append(np.array([c for c in "".join(parts)]))
    return sequences

# =============================================================================
def snake_case_prettify(s):
    return " ".join(w.capitalize() for w in s.split("_"))

# =============================================================================
def save_models(models):
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if not os.path.exists(modeldir):
        os.makedirs(modeldir)

    for name, model in models.items():
        model.save(os.path.join(modeldir, name))

# =============================================================================
def load_models():
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if os.path.exists(modeldir):
        return {f: tf.keras.models.load_model(os.path.join(modeldir, f)) for f in os.listdir(modeldir)}
    return {}

# =============================================================================
def print_diff_between_target_and_de_novo():

    dataset_dir = os.path.join(os.getcwd(), "newdatasets_p5A")
    
    de_novo_sequence = get_sequences(os.path.join(dataset_dir, "de_novo_sequence_p5A.txt"))[0]
    target_sequence = get_sequences(os.path.join(dataset_dir, "target_sequence_p5A.txt"))[0]

    gap_indices = np.where(de_novo_sequence == '-')[0]
    incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
    correct_indices = np.where(de_novo_sequence == target_sequence)[0]
    incorrect_non_gaps = np.setdiff1d(incorrect_indices, gap_indices)
    print(f"Length of target: {len(target_sequence)}")
    print(f"Number of incorrect non-gaps: {len(incorrect_non_gaps)}")
    print(f"Number of gaps: {len(gap_indices)}")

    print_sequence(de_novo_sequence, "Protein Scaffold", incorrect_indices, correct_indices)

# =============================================================================
def test_original_protein_scaffold():

    ut.mkdir_if_not_exists(RESULTS_PATH)
    ut.mkdir_if_not_exists(DATAPATH)

    epochs = 10
    percent_missings = np.array([0.10,0.09,0.08,0.07,0.06])
    num_gaps = np.array([4, 6, 8, 10])
    noise_percent = np.array([0.10,0.09,0.08,0.07,0.06])#0.25
    numgaps = np.array([4, 6, 8, 10])#5#just added today

    cwd = os.getcwd()

    # change to the location of the data for reading
    os.chdir(DATAPATH)
    de_novo_sequence = get_sequences("de_novo_sequence_p5A.txt")[0]
    target_sequence = get_sequences("target_sequence_p5A.txt")[0]
    alltrainingdata = get_sequences("training_sequences_p5A.txt")
    metadata = "training_sequences_metadata.csv"

    # then change to the results directory for writing
    os.chdir(RESULTS_PATH)

    sequences_to_train_on = [1000]
    # sequences_to_train_on = [100, 500, 1000, 2000]

    trainfilename = "transformer_breakdown_breakdown_of_numtrain.csv"
    headers = ["Num Training Instances", "Full Accuracy", "Gap Accuracy", "Nongap Accuracy"]
    with open(trainfilename, "w+", encoding="utf-8", newline="") as f:
        train_writer = csv.writer(f)
        train_writer.writerow(headers)

    train_file = open(trainfilename, "a+", encoding="utf-8", newline="")
    train_writer = csv.writer(train_file)

    for numtrain in sequences_to_train_on:

        trainingdata = alltrainingdata[:numtrain]
        random.shuffle(trainingdata)

        incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices = np.where(de_novo_sequence == target_sequence)[0]

        fixer = ProteinScaffoldFixer(trainingdata, noise_percent=noise_percent, epochs=epochs,numgaps=numgaps)#today
        weights = fixer.train(verbose=1)

        hist = fixer.history.history
        training_loss = hist["loss"]
        validation_loss = hist["val_loss"]
        training_acc = hist["accuracy"]
        validation_acc = hist["val_accuracy"]

        headers = ["Training Accuracy", "Validation Accuracy", "Training Loss", "Validation Loss"]
        with open(f"transformer_results_on_original_{numtrain}.csv", "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(zip(training_acc, validation_acc, training_loss, validation_loss))

        predict_only_gaps = False
        pred = fixer.predict_sequence(de_novo_sequence, predict_only_gaps)

        incorrect_indices = np.where(target_sequence != pred)[0]
        correct_indices = np.where(target_sequence == pred)[0]
        indices_to_predict = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices_to_predict = np.intersect1d(correct_indices, indices_to_predict)

        gap_indices = np.where(de_novo_sequence == "-")[0]
        error_indices = np.setdiff1d(indices_to_predict, gap_indices)

        correct_gap_indices = np.intersect1d(gap_indices, correct_indices)
        correct_error_indices = np.intersect1d(error_indices, correct_indices)

        gap_acc = len(correct_gap_indices) / len(gap_indices)
        err_acc = len(correct_error_indices) / len(error_indices)
        full_acc = len(np.where(target_sequence == pred)[0]) / len(target_sequence)

        # print to console the prediction
        # Print to console
        print(f"Gap Acc for Transformer: {gap_acc}")
        print(f"Err Acc for Transformer: {err_acc}")
        print(f"Full Acc for Transformer: {full_acc}")
        print_sequence(pred, f"Predicted for {numtrain}", incorrect_indices, correct_indices_to_predict)

        # write the final accuracies according to the number of training instances
        with open(trainfilename, "a+", encoding="utf-8", newline="") as f:
            train_writer = csv.writer(f)
            train_writer.writerow([numtrain, full_acc, gap_acc, err_acc])

        # and save the predictions as images for use in the paper
        ut.write_protein_scaffold_image(pred, incorrect_indices, correct_indices_to_predict, f"transformer_predictions_on_original_{numtrain}")
    
    
    train_file.close()
    os.chdir(cwd)
    return weights
#==============================================================================
from termcolor import colored

def print_comparison_with_colors(string1, string2, width=80):
    """
    Prints two strings with a fixed width for each line, adding a space after each character,
    and comparing the characters at each position. If they are not the same, it colors the character
    from string1 in red and bold.

    :param string1: The first input string to be printed and compared.
    :param string2: The second input string to be compared with the first.
    :param width: The number of characters in each line including spaces.
    """
    # Remove any existing spaces from the strings
    string1 = string1.replace(' ', '')
    string2 = string2.replace(' ', '')
    
    # Add a space after each character in the strings
    spaced_string1 = ' '.join(string1[i] for i in range(len(string1)))
    spaced_string2 = ' '.join(string2[i] for i in range(len(string2)))

    # Initialize the index for the substring
    start_index = 0
    
    # The actual width needs to be adjusted because each character now takes two positions
    # (one for the character and one for the space)
    adjusted_width = width // 2

    # Loop through the strings, taking substrings of the specified width and comparing them
    while start_index < len(spaced_string1):
        # End index is the start index plus the width or the end of the string, whichever comes first
        end_index = min(start_index + width, len(spaced_string1))
        # Initialize a list to hold the formatted characters
        formatted_line = []
        # Compare the characters from string1 and string2
        for i in range(start_index, end_index, 2):  # Step by 2 to skip spaces
            if i < len(spaced_string2) and spaced_string1[i] != spaced_string2[i]:
                # If characters don't match, append the red bold character from string1
                formatted_line.append(colored(spaced_string1[i], 'red', attrs=['bold']))
            else:
                # Otherwise, append the character from string1 as it is
                formatted_line.append(spaced_string1[i])
            # Append a space after each character
            formatted_line.append(' ')
        
        # Join the formatted characters and print the line
        print(''.join(formatted_line))
        # Increment the start_index to move to the next substring
        start_index += width
# =============================================================================

In [22]:
# Input strings provided by the user
target_seq_ = ('EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYDASTRATGIPDRFSGSGSGAD\
FLLTISSLEPEDFAMYYCQQYGRSPYTFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYP\
REAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQ\
GLSSPVTKSFNRGEC')

de_novo_seq_ = (
    '-IVLT---G--S--PG-RVTLSCR-S-SV-Y-Y-AWYQQK-G-AYRLLI-DA-TRATG-P-R-SG-G--ADWLLTI-SLEPEDF-\
    F-YCQQYGRSPYAFGPGTKEDILRPVAAPS---FPPSDEQLKS-SA-VQCL-QQFY-REA-VQT-V-RA-QSGNS---VTEQPS--\
    STY-LSS-L-LS-ADYE-H--YAC-VT-QGLS-WVTYSFNDGEC')

In [23]:
test_original_protein_scaffold()

Input shape: (764, 24)
Epoch 1/10
266/266 [==============================] - 157s 559ms/step - loss: 0.4913 - accuracy: 0.8489 - val_loss: 0.0921 - val_accuracy: 0.9733
Epoch 2/10
266/266 [==============================] - 148s 558ms/step - loss: 0.0900 - accuracy: 0.9741 - val_loss: 0.0495 - val_accuracy: 0.9849
Epoch 3/10
266/266 [==============================] - 149s 559ms/step - loss: 0.0657 - accuracy: 0.9806 - val_loss: 0.0410 - val_accuracy: 0.9874
Epoch 4/10
266/266 [==============================] - 149s 559ms/step - loss: 0.0580 - accuracy: 0.9827 - val_loss: 0.0366 - val_accuracy: 0.9885
Epoch 5/10
266/266 [==============================] - 149s 559ms/step - loss: 0.0533 - accuracy: 0.9839 - val_loss: 0.0336 - val_accuracy: 0.9894
Epoch 6/10
266/266 [==============================] - 149s 559ms/step - loss: 0.0501 - accuracy: 0.9848 - val_loss: 0.0318 - val_accuracy: 0.9898
Epoch 7/10
266/266 [==============================] - 149s 559ms/step - loss: 0.0474 - accuracy: 0.98

In [6]:
print_comparison_with_colors(de_novo_seq_, target_seq_)

- I V L T - - - G - - S - - P G - R V T L S C R - S - S V - Y - Y - A W Y Q Q K 
- G - A Y R L L I - D A - T R A T G - P - R - S G - G - - A D W L L T I - S L E 
P E D F - F - Y C Q Q Y G R S P Y A F G P G T K E D I L R P V A A P S - - - F P 
P S D E Q L K S - S A - V Q C L - Q Q F Y - R E A - V Q T - V - R A - Q S G N S 
- - - V T E Q P S - - S T Y - L S S - L - L S - A D Y E - H - - Y A C - V T - Q 
G L S - W V T Y S F N D G E C 


### Transformer Protein Scaffold Filling - p15

In [16]:
DATAPATH = os.path.join(os.getcwd(), "newdatasets_p15")
RESULTS_PATH = os.path.join(os.getcwd(), "transformer_results-p15")

In [17]:
'''
Trial summary
Hyperparameters:
conv1_filter_size: 10
conv2_filter_size: 9
conv1_filters: 50
conv2_filters: 66
bridge_filters: 120
max_pool: 2
dropout_: : 0.35
Score: 0.9727120995521545


Hyperparameters:
conv1_filter_size: 5
conv2_filter_size: 9
conv1_filters: 46
conv2_filters: 90
bridge_filters: 160
bridge_filter_size: 5
max_pool: 2
dropout_: : 0.5
Score: 0.9742878675460815
'''

# # =============================================================================
# class Encoder(layers.Layer):
#     """Encoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=10, conv2_filter_size=9, maxpool=2, dropout=0.25, name="encoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1D(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1D(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.maxpool = layers.MaxPooling1D(maxpool, padding="same")
#         self.dropout = layers.Dropout(dropout)
        

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.maxpool(self.conv2(self.dropout(self.maxpool(self.conv1(inputs))))))

# # =============================================================================
# class Decoder(layers.Layer):
#     """Decoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=5, conv2_filter_size=9, maxpool=2, dropout=0.25, name="decoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1DTranspose(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1DTranspose(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.upsample = layers.UpSampling1D(maxpool)
#         self.dropout = layers.Dropout(dropout)

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.upsample(self.conv2(self.dropout(self.upsample(self.conv1(inputs))))))

# # =============================================================================
# class Autoencoder(Model):
#     """Autoencoder"""
#     # -------------------------------------------------------------------------
#     def __init__(self, num_classes, name="autoencoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.num_classes = num_classes
#         self.hyperparameters = {
#             "conv1_filters": 46,
#             "conv2_filters": 90,
#             "conv1_filter_size": 5,
#             "conv2_filter_size": 9,
#             "bridge_filters": 160,
#             "bridge_filter_size": 5,
#             "dropout": 0.50,
#             "maxpool": 5,
#         }

#         hp = self.hyperparameters

#         self.encoder = Encoder(hp["conv1_filters"], hp["conv2_filters"])
#         self.bridge = layers.Conv1D(hp["bridge_filters"], hp["bridge_filter_size"], padding="same", activation="relu")
#         self.decoder = Decoder(hp["conv2_filters"], hp["conv1_filters"])
#         self.finallayer = layers.Conv1D(self.num_classes, hp["conv1_filter_size"], padding="same", activation="softmax")

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.finallayer(self.decoder(self.bridge(self.encoder(inputs))))

    
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class Transformer(Model):
    def __init__(self, input_shape_, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, 
                 mlp_dropout=0,name="transformers", **kwargs):
        super().__init__(name=name, **kwargs)
        self.input_shape_ = input_shape_
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout

    def transformer_encoder(self, inputs):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
        )(x, x)
        x = layers.Dropout(self.dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(self.dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    def build_model(self):
        print("Input shape:", self.input_shape_)
        inputs = tf.keras.Input(shape=self.input_shape_)
        x = inputs
        for _ in range(self.num_transformer_blocks):
            x = self.transformer_encoder(x)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in self.mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(self.mlp_dropout)(x)
        outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
        outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
        return models.Model(inputs, outputs)

#     def call(self, inputs):
#         x = inputs
#         for _ in range(self.num_transformer_blocks):
#             x = self.transformer_encoder(x)

#         x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
#         for dim in self.mlp_units:
#             x = layers.Dense(dim, activation="relu")(x)
#             x = layers.Dropout(self.mlp_dropout)(x)
#         outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
#         outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
#         return outputs
        
# =============================================================================
class PaddingType(Enum):
    ZERO=1
    TRUNCATE=2
    EMPTY=3

# =============================================================================

class NoisificationMethod(Enum):
    RANDOMSCATTER=1
    RANDOMCONTIG=2

weights = None

# =============================================================================
class ProteinScaffoldFixer():
    """Class to correct errors in a protein scaffold and fill gaps"""

    # -------------------------------------------------------------------------
    def __init__(self, 
                output_seqs, 
                paddingtype=PaddingType.EMPTY, 
                noise_percent=np.array([0.20, 0.30, 0.40]),#0.2, 
                noisemethod=NoisificationMethod.RANDOMSCATTER, 
                numgaps=np.array([4, 6, 8, 10]),#5, 
                mingapsize=3, 
                mincontigsize=1, 
                epochs=250,
                optimizer="adam",
                early_stopping=False):

        self.epochs = epochs
        self.numgaps = numgaps#added today
        self.optimizer = optimizer
        self.paddingtype = paddingtype
        self.noise_percent = noise_percent
        self.early_stopping = early_stopping

        self.classes = np.array(["-", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z'])
        self.ohe = OneHotEncoder(sparse_output=False, categories=[self.classes])

        self.max_seq_length = max((len(seq) for seq in output_seqs))
        self.maxpool = 2
        while self.max_seq_length % (self.maxpool * self.maxpool) != 0:
            self.max_seq_length += 1

        self.output_seqs = output_seqs
        #self.input_seqs = self.noisify_sequences(output_seqs, noise_percent, noisemethod, numgaps, mingapsize, mincontigsize)
        #########################
        self.output_seqs_all = []
        self.input_seqs_all = []
        
        for n_gaps in numgaps:
            for p_missing in noise_percent:
                self.output_seqs_temp = output_seqs
                self.input_seqs_temp = self.noisify_sequences(self.output_seqs_temp, p_missing, noisemethod, n_gaps, mingapsize=3, mincontigsize=1)
                self.output_seqs_all += self.output_seqs_temp
                self.input_seqs_all += self.input_seqs_temp
        #########################
        self.train_x = self.preprocess_sequences(self.input_seqs_all)
        self.train_y = self.preprocess_sequences(self.output_seqs_all)

        self.history = None

        #self.autoencoder = Autoencoder(len(self.classes))
        ### my additions ######
        self.input_shape = self.train_x.shape[1:]#(440,24)
        self.head_size=256
        self.num_heads=4
        self.ff_dim=4
        self.num_transformer_blocks=4#3
        self.mlp_units=[512,128]
        self.mlp_dropout=0.4
        self.dropout=0.25
        self.transformer = Transformer(self.input_shape, self.head_size, self.num_heads, self.ff_dim, 
                                self.num_transformer_blocks, self.mlp_units, self.dropout, self.mlp_dropout)
        #self.transformer = self.transformer.build_model()
    # =============================================================================
    def noisify_sequences(self, seqs, noise_percent, noisemethod, numgaps=5, mingapsize=3, mincontigsize=3):
        if noisemethod == NoisificationMethod.RANDOMSCATTER:
            return self.noisify_by_random_scatter(seqs, noise_percent)
        elif noisemethod == NoisificationMethod.RANDOMCONTIG:
            return self.noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize)

    # =============================================================================
    def noisify_by_random_scatter(self, seqs, noise_percent):
        sequences = deepcopy(seqs)
        # To noisify our input data, we will replace random amino acids with something else
        for seq in sequences:

            # We randomly sample from all the possible indices of seq
            indices_to_replace = random.sample(range(len(seq)), int(noise_percent * len(seq)))
            seq[indices_to_replace] = "-"

        return sequences

    # =============================================================================
    def noisify_by_random_contigs(self, seqs, noise_percent, numgaps, mingapsize, mincontigsize):

        sequences = deepcopy(seqs)
        for seqind, seq in enumerate(sequences):

            amino_acid_length = len(seq)
            amino_acids_to_replace = int(amino_acid_length * noise_percent)

            # The idea is to build a gap queue, randomly putting each amino acid to replace into each gap
            # Then, we build a contig queue, randomly putting each amino acid into each contig
            gap_queue = np.zeros(numgaps).astype(int)
            for _ in range(amino_acids_to_replace):

                # Any gap can be considered
                valid_gaps = np.arange(numgaps)

                # ... so long as we have no underfilled gaps
                underfilled = gap_queue < mingapsize
                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_gaps = valid_gaps[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                gap_queue[np.random.choice(valid_gaps)] += 1

            # There can always be one more contig than gaps (if there's a contig at beginnning and end of sequence)
            contig_queue = np.zeros(numgaps+1).astype(int)

            # We have to allocate all the amino acids NOT in gaps into contigs BETWEEN the gaps
            for _ in range(amino_acid_length - amino_acids_to_replace):

                # Any contig can be considered
                valid_contigs = np.arange(len(contig_queue))

                # ... so long as we have no underfilled contigs
                underfilled = contig_queue < mincontigsize

                # The exceptions are the first and last contigs. They are never considered underfilled
                underfilled[0] = False
                underfilled[-1] = False

                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_contigs = valid_contigs[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                contig_queue[np.random.choice(valid_contigs)] += 1
            
            # Once we have determined gap_queue and contig_queue, we iterate over them to set the gaps equal
            # to our blank amino acid character

            sequence_pointer = 0
            iscontig=True
            while sequence_pointer < len(seq):
                if iscontig:
                    # Don't do anything for the contig except increment the pointer and pop off the contig queue
                    sequence_pointer += contig_queue[0]
                    contig_queue = np.delete(contig_queue, 0)
                else:
                    seq[sequence_pointer:sequence_pointer+gap_queue[0]] = '-'
                    sequence_pointer += gap_queue[0]
                    gap_queue = np.delete(gap_queue, 0)

                # We alternate between contigs and gaps
                iscontig = not iscontig

        return sequences

    # -------------------------------------------------------------------------
    def predict_sequence(self, seq, predict_only_gaps):

        scaffold = self.preprocess_sequences([seq])
        #pred = self.autoencoder.predict(scaffold).reshape(self.max_seq_length, len(self.classes))
        pred = self.transformer.predict(scaffold).reshape(self.max_seq_length, len(self.classes))

        # Set the probability of empty "-" to zero, since we always want to predict something
        emptyclass = np.where(self.ohe.transform(np.array("-").reshape(-1, 1))[0])[0][0]
        pred[:, emptyclass] = 0.0

        # Convert the probability distribution to a one-hot encoded vector
        mask = pred == np.amax(pred, axis=1).reshape(pred.shape[0], 1)
        indices = list((i, np.where(mask[i])[0][0]) for i in range(mask.shape[0]))

        pred = np.zeros(pred.shape)
        for i in indices:
            pred[i] = 1

        # Then we can use our one hot encoder to convert back to the original sequence of classes
        pred = self.ohe.inverse_transform(pred[:len(seq), :len(self.classes)]).reshape(len(seq))

        if predict_only_gaps:
            # We only care about predicting the gaps in seq, so replace amino acids in prediction with original nongaps
            nongaps = np.where(seq != '-')[0]
            pred[nongaps] = seq[nongaps]

        return pred

    # =============================================================================
    def preprocess_sequences(self, seqs: List[np.array]) -> np.array:

        # the value -1 lets numpy know to infer the shape. So it's just a column vector of length num_samples
        seqs = [np.array(seq).reshape(-1, 1) for seq in seqs]

        # One-hot encode each sequence
        seqs = [self.ohe.fit_transform(seq) for seq in seqs]

        if self.paddingtype == PaddingType.ZERO:
            # The sequences may have different lengths, so we will pad them with zeros
            # We are padding to fill up to max length, then we can turn it into a single numpy tensor
            return np.array([np.pad(seq, ((0, self.max_seq_length-len(seq)), (0, 0))) for seq in seqs])

        elif self.paddingtype == PaddingType.EMPTY:

            # We pad with the "empty" class
            emptyclass = self.ohe.fit_transform(np.array("-").reshape(-1, 1))[0]

            # dynamically extend each seq by enough emptyvals to make a single sequence length
            return np.array([np.vstack((seq, *(emptyclass for _ in range(self.max_seq_length-len(seq))))) for seq in seqs])

        elif self.paddingtype == PaddingType.TRUNCATE:
            # FIXME: Implement me
            raise Exception("Have not implemented Padding Type TRUNCATE!")

    # =============================================================================
    def train(self, verbose="auto", optimizer="adam", epochs=None):
        self.optimizer = optimizer
        if epochs:
            self.epochs = epochs

#         self.autoencoder.compile(optimizer=self.optimizer, loss="categorical_crossentropy",
#                                  metrics=["accuracy"])
        self.transformer = self.transformer.build_model()
        self.transformer.compile(loss="categorical_crossentropy",
                                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                                    metrics=["accuracy"],)
        callbacks = []
        if self.early_stopping:
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10))
        #self.history = self.autoencoder.fit(self.train_x, self.train_y, epochs=self.epochs, validation_split=0.15, verbose=verbose, callbacks=callbacks)
        self.history = self.transformer.fit(self.train_x, self.train_y, epochs=self.epochs, batch_size=64,
                                            validation_split=0.30, verbose=verbose, callbacks=callbacks)
        # Save the history to a file using pickle
        with open('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-p15/model_p5A/model_history.pkl', 'wb') as file:
            pk.dump(self.history, file)
        #plot_model(self.transformer, to_file='transformer_plot.png', show_shapes=True, show_layer_names=True)
        #print(self.transformer.summary())
        self.transformer.save('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-p15/model_p5A/transormer_model_p5A.h5')
        
# =============================================================================
def highlight_indices(seq: np.array, indices: np.array, color: str):
    # We use deepcopy to prevent mutation and we cast to object so that we can treat contents as python strings
    # otherwise, it gets messed up as it treats each element as a single character
    newseq = deepcopy(seq).astype('object')
    newseq[indices] = np.vectorize(lambda x: colored(x, color, attrs=["bold"]))(seq[indices])
    return newseq

# =============================================================================
def print_sequence(seq, 
                   header: str=None, 
                   incorrect_indices: Optional[np.array]=None, 
                   correct_indices: Optional[np.array]=None):

    newseq = deepcopy(seq)
    if correct_indices is not None and correct_indices.size != 0:
        newseq = highlight_indices(newseq, correct_indices, "green")
    if incorrect_indices is not None and incorrect_indices.size != 0:
        newseq = highlight_indices(newseq, incorrect_indices, "red")

    line_length = 40
    if header:
        print(header)
    print("=" * line_length)

    i = 0
    while i < len(newseq):
        print(" ".join(newseq[i: i+line_length]))
        i += line_length

# =============================================================================
def get_sequences(fasta_file: str) -> List[np.array]:
    sequences = []
    lines = []
    with open(fasta_file, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append(np.array([c for c in "".join(parts)]))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append(np.array([c for c in "".join(parts)]))
    return sequences

# =============================================================================
def snake_case_prettify(s):
    return " ".join(w.capitalize() for w in s.split("_"))

# =============================================================================
def save_models(models):
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if not os.path.exists(modeldir):
        os.makedirs(modeldir)

    for name, model in models.items():
        model.save(os.path.join(modeldir, name))

# =============================================================================
def load_models():
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if os.path.exists(modeldir):
        return {f: tf.keras.models.load_model(os.path.join(modeldir, f)) for f in os.listdir(modeldir)}
    return {}

# =============================================================================
def print_diff_between_target_and_de_novo():

    dataset_dir = os.path.join(os.getcwd(), "newdatasets_p15")
    
    de_novo_sequence = get_sequences(os.path.join(dataset_dir, "de_novo_sequence_p15.txt"))[0]
    target_sequence = get_sequences(os.path.join(dataset_dir, "target_sequence_p15.txt"))[0]

    gap_indices = np.where(de_novo_sequence == '-')[0]
    incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
    correct_indices = np.where(de_novo_sequence == target_sequence)[0]
    incorrect_non_gaps = np.setdiff1d(incorrect_indices, gap_indices)
    print(f"Length of target: {len(target_sequence)}")
    print(f"Number of incorrect non-gaps: {len(incorrect_non_gaps)}")
    print(f"Number of gaps: {len(gap_indices)}")

    print_sequence(de_novo_sequence, "Protein Scaffold", incorrect_indices, correct_indices)

# =============================================================================
def test_original_protein_scaffold():

    ut.mkdir_if_not_exists(RESULTS_PATH)
    ut.mkdir_if_not_exists(DATAPATH)

    epochs = 10
    percent_missings = np.array([0.10,0.09,0.08,0.07,0.06])
    num_gaps = np.array([4, 6, 8, 10])
    noise_percent = np.array([0.10,0.09,0.08,0.07,0.06])#0.25
    numgaps = np.array([4, 6, 8, 10])#5#just added today

    cwd = os.getcwd()

    # change to the location of the data for reading
    os.chdir(DATAPATH)
    de_novo_sequence = get_sequences("de_novo_sequence_p15.txt")[0]
    target_sequence = get_sequences("target_sequence_p15.txt")[0]
    alltrainingdata = get_sequences("training_sequences_p15.txt")
    metadata = "training_sequences_metadata.csv"

    # then change to the results directory for writing
    os.chdir(RESULTS_PATH)

    sequences_to_train_on = [1000]
    # sequences_to_train_on = [100, 500, 1000, 2000]

    trainfilename = "transformer_breakdown_breakdown_of_numtrain.csv"
    headers = ["Num Training Instances", "Full Accuracy", "Gap Accuracy", "Nongap Accuracy"]
    with open(trainfilename, "w+", encoding="utf-8", newline="") as f:
        train_writer = csv.writer(f)
        train_writer.writerow(headers)

    train_file = open(trainfilename, "a+", encoding="utf-8", newline="")
    train_writer = csv.writer(train_file)

    for numtrain in sequences_to_train_on:

        trainingdata = alltrainingdata[:numtrain]
        random.shuffle(trainingdata)

        incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices = np.where(de_novo_sequence == target_sequence)[0]

        fixer = ProteinScaffoldFixer(trainingdata, noise_percent=noise_percent, epochs=epochs,numgaps=numgaps)#today
        weights = fixer.train(verbose=1)

        hist = fixer.history.history
        training_loss = hist["loss"]
        validation_loss = hist["val_loss"]
        training_acc = hist["accuracy"]
        validation_acc = hist["val_accuracy"]

        headers = ["Training Accuracy", "Validation Accuracy", "Training Loss", "Validation Loss"]
        with open(f"transformer_results_on_original_{numtrain}.csv", "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(zip(training_acc, validation_acc, training_loss, validation_loss))

        predict_only_gaps = False
        pred = fixer.predict_sequence(de_novo_sequence, predict_only_gaps)

        incorrect_indices = np.where(target_sequence != pred)[0]
        correct_indices = np.where(target_sequence == pred)[0]
        indices_to_predict = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices_to_predict = np.intersect1d(correct_indices, indices_to_predict)

        gap_indices = np.where(de_novo_sequence == "-")[0]
        error_indices = np.setdiff1d(indices_to_predict, gap_indices)

        correct_gap_indices = np.intersect1d(gap_indices, correct_indices)
        correct_error_indices = np.intersect1d(error_indices, correct_indices)

        gap_acc = len(correct_gap_indices) / len(gap_indices)
        err_acc = len(correct_error_indices) / len(error_indices)
        full_acc = len(np.where(target_sequence == pred)[0]) / len(target_sequence)

        # print to console the prediction
        # Print to console
        print(f"Gap Acc for Transformer: {gap_acc}")
        print(f"Err Acc for Transformer: {err_acc}")
        print(f"Full Acc for Transformer: {full_acc}")
        print_sequence(pred, f"Predicted for {numtrain}", incorrect_indices, correct_indices_to_predict)

        # write the final accuracies according to the number of training instances
        with open(trainfilename, "a+", encoding="utf-8", newline="") as f:
            train_writer = csv.writer(f)
            train_writer.writerow([numtrain, full_acc, gap_acc, err_acc])

        # and save the predictions as images for use in the paper
        ut.write_protein_scaffold_image(pred, incorrect_indices, correct_indices_to_predict, f"transformer_predictions_on_original_{numtrain}")
    
    
    train_file.close()
    os.chdir(cwd)
    return weights
#==============================================================================
from termcolor import colored

def print_comparison_with_colors(string1, string2, width=80):
    """
    Prints two strings with a fixed width for each line, adding a space after each character,
    and comparing the characters at each position. If they are not the same, it colors the character
    from string1 in red and bold.

    :param string1: The first input string to be printed and compared.
    :param string2: The second input string to be compared with the first.
    :param width: The number of characters in each line including spaces.
    """
    # Remove any existing spaces from the strings
    string1 = string1.replace(' ', '')
    string2 = string2.replace(' ', '')
    
    # Add a space after each character in the strings
    spaced_string1 = ' '.join(string1[i] for i in range(len(string1)))
    spaced_string2 = ' '.join(string2[i] for i in range(len(string2)))

    # Initialize the index for the substring
    start_index = 0
    
    # The actual width needs to be adjusted because each character now takes two positions
    # (one for the character and one for the space)
    adjusted_width = width // 2

    # Loop through the strings, taking substrings of the specified width and comparing them
    while start_index < len(spaced_string1):
        # End index is the start index plus the width or the end of the string, whichever comes first
        end_index = min(start_index + width, len(spaced_string1))
        # Initialize a list to hold the formatted characters
        formatted_line = []
        # Compare the characters from string1 and string2
        for i in range(start_index, end_index, 2):  # Step by 2 to skip spaces
            if i < len(spaced_string2) and spaced_string1[i] != spaced_string2[i]:
                # If characters don't match, append the red bold character from string1
                formatted_line.append(colored(spaced_string1[i], 'red', attrs=['bold']))
            else:
                # Otherwise, append the character from string1 as it is
                formatted_line.append(spaced_string1[i])
            # Append a space after each character
            formatted_line.append(' ')
        
        # Join the formatted characters and print the line
        print(''.join(formatted_line))
        # Increment the start_index to move to the next substring
        start_index += width
# =============================================================================


In [18]:
# Input strings provided by the user
target_seq_ = ('DIQMTQSPSTLSASVGDAVTITCRASQSLNVWLAWYQQKPGKPPKLLIYE\
ASNLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYPYTFGQ\
GAKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKV\
DNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKLYACEVTHQG\
LSSPVTKSFNRGEC')

de_novo_seq_ = (
    'DI--LQSPSTLSASV--AVTI-CRA-QSLNQ-L-WYQQKPGKPPKLLIYEA-NPESGVSSRFSGSGSGTE--LT-SSLQPDDFA----QQY-SYP-TFGQ\
-AKL-I-RTVAAP---MFPPS-EQLKCGTNS-Y-T-N-FY--EAKV-W-VDL----G-SQES-TEQDSKDSTNS-S-TLTLSKGDYEKH-L-A-EV-HN-\
L-SPVTK--NR---')

In [19]:
test_original_protein_scaffold()

Input shape: (452, 24)
Epoch 1/10
219/219 [==============================] - 67s 267ms/step - loss: 0.6024 - accuracy: 0.8234 - val_loss: 0.1465 - val_accuracy: 0.9584
Epoch 2/10
219/219 [==============================] - 58s 267ms/step - loss: 0.1409 - accuracy: 0.9599 - val_loss: 0.0890 - val_accuracy: 0.9738
Epoch 3/10
219/219 [==============================] - 58s 267ms/step - loss: 0.1083 - accuracy: 0.9687 - val_loss: 0.0751 - val_accuracy: 0.9770
Epoch 4/10
219/219 [==============================] - 59s 268ms/step - loss: 0.0951 - accuracy: 0.9718 - val_loss: 0.0659 - val_accuracy: 0.9793
Epoch 5/10
219/219 [==============================] - 59s 268ms/step - loss: 0.0892 - accuracy: 0.9730 - val_loss: 0.0606 - val_accuracy: 0.9806
Epoch 6/10
219/219 [==============================] - 59s 267ms/step - loss: 0.0843 - accuracy: 0.9742 - val_loss: 0.0570 - val_accuracy: 0.9813
Epoch 7/10
219/219 [==============================] - 59s 268ms/step - loss: 0.0797 - accuracy: 0.9754 - va

In [18]:
print_comparison_with_colors(de_novo_seq_, target_seq_)

D I - - L Q S P S T L S A S V - - A V T I - C R A - Q S L N Q - L - W Y Q Q K P 
G K P P K L L I Y E A - N P E S G V S S R F S G S G S G T E - - L T - S S L Q P 
D D F A - - - - Q Q Y - S Y P - T F G Q - A K L - I - R T V A A P - - - M F P P 
S - E Q L K C G T N S - Y - T - N - F Y - - E A K V - W - V D L - - - - G - S Q 
E S - T E Q D S K D S T N S - S - T L T L S K G D Y E K H - L - A - E V - H N - 
L - S P V T K - - N R - - - 


### Transformer Protein Scaffold Filling - p18

In [2]:
DATAPATH = os.path.join(os.getcwd(), "newdatasets_p18")
RESULTS_PATH = os.path.join(os.getcwd(), "transformer_results-p18")

In [3]:
'''
Trial summary
Hyperparameters:
conv1_filter_size: 10
conv2_filter_size: 9
conv1_filters: 50
conv2_filters: 66
bridge_filters: 120
max_pool: 2
dropout_: : 0.35
Score: 0.9727120995521545


Hyperparameters:
conv1_filter_size: 5
conv2_filter_size: 9
conv1_filters: 46
conv2_filters: 90
bridge_filters: 160
bridge_filter_size: 5
max_pool: 2
dropout_: : 0.5
Score: 0.9742878675460815
'''

# # =============================================================================
# class Encoder(layers.Layer):
#     """Encoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=10, conv2_filter_size=9, maxpool=2, dropout=0.25, name="encoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1D(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1D(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.maxpool = layers.MaxPooling1D(maxpool, padding="same")
#         self.dropout = layers.Dropout(dropout)
        

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.maxpool(self.conv2(self.dropout(self.maxpool(self.conv1(inputs))))))

# # =============================================================================
# class Decoder(layers.Layer):
#     """Decoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=5, conv2_filter_size=9, maxpool=2, dropout=0.25, name="decoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1DTranspose(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1DTranspose(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.upsample = layers.UpSampling1D(maxpool)
#         self.dropout = layers.Dropout(dropout)

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.upsample(self.conv2(self.dropout(self.upsample(self.conv1(inputs))))))

# # =============================================================================
# class Autoencoder(Model):
#     """Autoencoder"""
#     # -------------------------------------------------------------------------
#     def __init__(self, num_classes, name="autoencoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.num_classes = num_classes
#         self.hyperparameters = {
#             "conv1_filters": 46,
#             "conv2_filters": 90,
#             "conv1_filter_size": 5,
#             "conv2_filter_size": 9,
#             "bridge_filters": 160,
#             "bridge_filter_size": 5,
#             "dropout": 0.50,
#             "maxpool": 5,
#         }

#         hp = self.hyperparameters

#         self.encoder = Encoder(hp["conv1_filters"], hp["conv2_filters"])
#         self.bridge = layers.Conv1D(hp["bridge_filters"], hp["bridge_filter_size"], padding="same", activation="relu")
#         self.decoder = Decoder(hp["conv2_filters"], hp["conv1_filters"])
#         self.finallayer = layers.Conv1D(self.num_classes, hp["conv1_filter_size"], padding="same", activation="softmax")

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.finallayer(self.decoder(self.bridge(self.encoder(inputs))))

    
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class Transformer(Model):
    def __init__(self, input_shape_, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, 
                 mlp_dropout=0,name="transformers", **kwargs):
        super().__init__(name=name, **kwargs)
        self.input_shape_ = input_shape_
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout

    def transformer_encoder(self, inputs):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
        )(x, x)
        x = layers.Dropout(self.dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(self.dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    def build_model(self):
        print("Input shape:", self.input_shape_)
        inputs = tf.keras.Input(shape=self.input_shape_)
        x = inputs
        for _ in range(self.num_transformer_blocks):
            x = self.transformer_encoder(x)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in self.mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(self.mlp_dropout)(x)
        outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
        outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
        return models.Model(inputs, outputs)

#     def call(self, inputs):
#         x = inputs
#         for _ in range(self.num_transformer_blocks):
#             x = self.transformer_encoder(x)

#         x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
#         for dim in self.mlp_units:
#             x = layers.Dense(dim, activation="relu")(x)
#             x = layers.Dropout(self.mlp_dropout)(x)
#         outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
#         outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
#         return outputs
        
# =============================================================================
class PaddingType(Enum):
    ZERO=1
    TRUNCATE=2
    EMPTY=3

# =============================================================================

class NoisificationMethod(Enum):
    RANDOMSCATTER=1
    RANDOMCONTIG=2

weights = None

# =============================================================================
class ProteinScaffoldFixer():
    """Class to correct errors in a protein scaffold and fill gaps"""

    # -------------------------------------------------------------------------
    def __init__(self, 
                output_seqs, 
                paddingtype=PaddingType.EMPTY, 
                noise_percent=np.array([0.20, 0.30, 0.40]),#0.2, 
                noisemethod=NoisificationMethod.RANDOMSCATTER, 
                numgaps=np.array([4, 6, 8, 10]),#5, 
                mingapsize=3, 
                mincontigsize=1, 
                epochs=250,
                optimizer="adam",
                early_stopping=False):

        self.epochs = epochs
        self.numgaps = numgaps#added today
        self.optimizer = optimizer
        self.paddingtype = paddingtype
        self.noise_percent = noise_percent
        self.early_stopping = early_stopping

        self.classes = np.array(["-", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z'])
        self.ohe = OneHotEncoder(sparse_output=False, categories=[self.classes])

        self.max_seq_length = max((len(seq) for seq in output_seqs))
        self.maxpool = 2
        while self.max_seq_length % (self.maxpool * self.maxpool) != 0:
            self.max_seq_length += 1

        self.output_seqs = output_seqs
        #self.input_seqs = self.noisify_sequences(output_seqs, noise_percent, noisemethod, numgaps, mingapsize, mincontigsize)
        #########################
        self.output_seqs_all = []
        self.input_seqs_all = []
        
        for n_gaps in numgaps:
            for p_missing in noise_percent:
                self.output_seqs_temp = output_seqs
                self.input_seqs_temp = self.noisify_sequences(self.output_seqs_temp, p_missing, noisemethod, n_gaps, mingapsize=3, mincontigsize=1)
                self.output_seqs_all += self.output_seqs_temp
                self.input_seqs_all += self.input_seqs_temp
        #########################
        self.train_x = self.preprocess_sequences(self.input_seqs_all)
        self.train_y = self.preprocess_sequences(self.output_seqs_all)

        self.history = None

        #self.autoencoder = Autoencoder(len(self.classes))
        ### my additions ######
        self.input_shape = self.train_x.shape[1:]#(440,24)
        self.head_size=256
        self.num_heads=4
        self.ff_dim=4
        self.num_transformer_blocks=4#3
        self.mlp_units=[512,128]
        self.mlp_dropout=0.4
        self.dropout=0.25
        self.transformer = Transformer(self.input_shape, self.head_size, self.num_heads, self.ff_dim, 
                                self.num_transformer_blocks, self.mlp_units, self.dropout, self.mlp_dropout)
        #self.transformer = self.transformer.build_model()
    # =============================================================================
    def noisify_sequences(self, seqs, noise_percent, noisemethod, numgaps=5, mingapsize=3, mincontigsize=3):
        if noisemethod == NoisificationMethod.RANDOMSCATTER:
            return self.noisify_by_random_scatter(seqs, noise_percent)
        elif noisemethod == NoisificationMethod.RANDOMCONTIG:
            return self.noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize)

    # =============================================================================
    def noisify_by_random_scatter(self, seqs, noise_percent):
        sequences = deepcopy(seqs)
        # To noisify our input data, we will replace random amino acids with something else
        for seq in sequences:

            # We randomly sample from all the possible indices of seq
            indices_to_replace = random.sample(range(len(seq)), int(noise_percent * len(seq)))
            seq[indices_to_replace] = "-"

        return sequences

    # =============================================================================
    def noisify_by_random_contigs(self, seqs, noise_percent, numgaps, mingapsize, mincontigsize):

        sequences = deepcopy(seqs)
        for seqind, seq in enumerate(sequences):

            amino_acid_length = len(seq)
            amino_acids_to_replace = int(amino_acid_length * noise_percent)

            # The idea is to build a gap queue, randomly putting each amino acid to replace into each gap
            # Then, we build a contig queue, randomly putting each amino acid into each contig
            gap_queue = np.zeros(numgaps).astype(int)
            for _ in range(amino_acids_to_replace):

                # Any gap can be considered
                valid_gaps = np.arange(numgaps)

                # ... so long as we have no underfilled gaps
                underfilled = gap_queue < mingapsize
                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_gaps = valid_gaps[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                gap_queue[np.random.choice(valid_gaps)] += 1

            # There can always be one more contig than gaps (if there's a contig at beginnning and end of sequence)
            contig_queue = np.zeros(numgaps+1).astype(int)

            # We have to allocate all the amino acids NOT in gaps into contigs BETWEEN the gaps
            for _ in range(amino_acid_length - amino_acids_to_replace):

                # Any contig can be considered
                valid_contigs = np.arange(len(contig_queue))

                # ... so long as we have no underfilled contigs
                underfilled = contig_queue < mincontigsize

                # The exceptions are the first and last contigs. They are never considered underfilled
                underfilled[0] = False
                underfilled[-1] = False

                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_contigs = valid_contigs[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                contig_queue[np.random.choice(valid_contigs)] += 1
            
            # Once we have determined gap_queue and contig_queue, we iterate over them to set the gaps equal
            # to our blank amino acid character

            sequence_pointer = 0
            iscontig=True
            while sequence_pointer < len(seq):
                if iscontig:
                    # Don't do anything for the contig except increment the pointer and pop off the contig queue
                    sequence_pointer += contig_queue[0]
                    contig_queue = np.delete(contig_queue, 0)
                else:
                    seq[sequence_pointer:sequence_pointer+gap_queue[0]] = '-'
                    sequence_pointer += gap_queue[0]
                    gap_queue = np.delete(gap_queue, 0)

                # We alternate between contigs and gaps
                iscontig = not iscontig

        return sequences

    # -------------------------------------------------------------------------
    def predict_sequence(self, seq, predict_only_gaps):

        scaffold = self.preprocess_sequences([seq])
        #pred = self.autoencoder.predict(scaffold).reshape(self.max_seq_length, len(self.classes))
        pred = self.transformer.predict(scaffold).reshape(self.max_seq_length, len(self.classes))

        # Set the probability of empty "-" to zero, since we always want to predict something
        emptyclass = np.where(self.ohe.transform(np.array("-").reshape(-1, 1))[0])[0][0]
        pred[:, emptyclass] = 0.0

        # Convert the probability distribution to a one-hot encoded vector
        mask = pred == np.amax(pred, axis=1).reshape(pred.shape[0], 1)
        indices = list((i, np.where(mask[i])[0][0]) for i in range(mask.shape[0]))

        pred = np.zeros(pred.shape)
        for i in indices:
            pred[i] = 1

        # Then we can use our one hot encoder to convert back to the original sequence of classes
        pred = self.ohe.inverse_transform(pred[:len(seq), :len(self.classes)]).reshape(len(seq))

        if predict_only_gaps:
            # We only care about predicting the gaps in seq, so replace amino acids in prediction with original nongaps
            nongaps = np.where(seq != '-')[0]
            pred[nongaps] = seq[nongaps]

        return pred

    # =============================================================================
    def preprocess_sequences(self, seqs: List[np.array]) -> np.array:

        # the value -1 lets numpy know to infer the shape. So it's just a column vector of length num_samples
        seqs = [np.array(seq).reshape(-1, 1) for seq in seqs]

        # One-hot encode each sequence
        seqs = [self.ohe.fit_transform(seq) for seq in seqs]

        if self.paddingtype == PaddingType.ZERO:
            # The sequences may have different lengths, so we will pad them with zeros
            # We are padding to fill up to max length, then we can turn it into a single numpy tensor
            return np.array([np.pad(seq, ((0, self.max_seq_length-len(seq)), (0, 0))) for seq in seqs])

        elif self.paddingtype == PaddingType.EMPTY:

            # We pad with the "empty" class
            emptyclass = self.ohe.fit_transform(np.array("-").reshape(-1, 1))[0]

            # dynamically extend each seq by enough emptyvals to make a single sequence length
            return np.array([np.vstack((seq, *(emptyclass for _ in range(self.max_seq_length-len(seq))))) for seq in seqs])

        elif self.paddingtype == PaddingType.TRUNCATE:
            # FIXME: Implement me
            raise Exception("Have not implemented Padding Type TRUNCATE!")

    # =============================================================================
    def train(self, verbose="auto", optimizer="adam", epochs=None):
        self.optimizer = optimizer
        if epochs:
            self.epochs = epochs

#         self.autoencoder.compile(optimizer=self.optimizer, loss="categorical_crossentropy",
#                                  metrics=["accuracy"])
        self.transformer = self.transformer.build_model()
        self.transformer.compile(loss="categorical_crossentropy",
                                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                                    metrics=["accuracy"],)
        callbacks = []
        if self.early_stopping:
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10))
        #self.history = self.autoencoder.fit(self.train_x, self.train_y, epochs=self.epochs, validation_split=0.15, verbose=verbose, callbacks=callbacks)
        self.history = self.transformer.fit(self.train_x, self.train_y, epochs=self.epochs, batch_size=64,
                                            validation_split=0.30, verbose=verbose, callbacks=callbacks)
        # Save the history to a file using pickle
        with open('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-p18/model_p5A/model_history.pkl', 'wb') as file:
            pk.dump(self.history, file)
        #plot_model(self.transformer, to_file='transformer_plot.png', show_shapes=True, show_layer_names=True)
        #print(self.transformer.summary())
        self.transformer.save('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-p18/model_p5A/transormer_model_p5A.h5')
        
# =============================================================================
def highlight_indices(seq: np.array, indices: np.array, color: str):
    # We use deepcopy to prevent mutation and we cast to object so that we can treat contents as python strings
    # otherwise, it gets messed up as it treats each element as a single character
    newseq = deepcopy(seq).astype('object')
    newseq[indices] = np.vectorize(lambda x: colored(x, color, attrs=["bold"]))(seq[indices])
    return newseq

# =============================================================================
def print_sequence(seq, 
                   header: str=None, 
                   incorrect_indices: Optional[np.array]=None, 
                   correct_indices: Optional[np.array]=None):

    newseq = deepcopy(seq)
    if correct_indices is not None and correct_indices.size != 0:
        newseq = highlight_indices(newseq, correct_indices, "green")
    if incorrect_indices is not None and incorrect_indices.size != 0:
        newseq = highlight_indices(newseq, incorrect_indices, "red")

    line_length = 40
    if header:
        print(header)
    print("=" * line_length)

    i = 0
    while i < len(newseq):
        print(" ".join(newseq[i: i+line_length]))
        i += line_length

# =============================================================================
def get_sequences(fasta_file: str) -> List[np.array]:
    sequences = []
    lines = []
    with open(fasta_file, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append(np.array([c for c in "".join(parts)]))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append(np.array([c for c in "".join(parts)]))
    return sequences

# =============================================================================
def snake_case_prettify(s):
    return " ".join(w.capitalize() for w in s.split("_"))

# =============================================================================
def save_models(models):
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if not os.path.exists(modeldir):
        os.makedirs(modeldir)

    for name, model in models.items():
        model.save(os.path.join(modeldir, name))

# =============================================================================
def load_models():
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if os.path.exists(modeldir):
        return {f: tf.keras.models.load_model(os.path.join(modeldir, f)) for f in os.listdir(modeldir)}
    return {}

# =============================================================================
def print_diff_between_target_and_de_novo():

    dataset_dir = os.path.join(os.getcwd(), "newdatasets_p18")
    
    de_novo_sequence = get_sequences(os.path.join(dataset_dir, "de_novo_sequence_p18.txt"))[0]
    target_sequence = get_sequences(os.path.join(dataset_dir, "target_sequence_p18.txt"))[0]

    gap_indices = np.where(de_novo_sequence == '-')[0]
    incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
    correct_indices = np.where(de_novo_sequence == target_sequence)[0]
    incorrect_non_gaps = np.setdiff1d(incorrect_indices, gap_indices)
    print(f"Length of target: {len(target_sequence)}")
    print(f"Number of incorrect non-gaps: {len(incorrect_non_gaps)}")
    print(f"Number of gaps: {len(gap_indices)}")

    print_sequence(de_novo_sequence, "Protein Scaffold", incorrect_indices, correct_indices)

# =============================================================================
def test_original_protein_scaffold():

    ut.mkdir_if_not_exists(RESULTS_PATH)
    ut.mkdir_if_not_exists(DATAPATH)

    epochs = 10
    percent_missings = np.array([0.10,0.09,0.08,0.07,0.06])
    num_gaps = np.array([4, 6, 8, 10])
    noise_percent = np.array([0.10,0.09,0.08,0.07,0.06])#0.25
    numgaps = np.array([4, 6, 8, 10])#5#just added today

    cwd = os.getcwd()

    # change to the location of the data for reading
    os.chdir(DATAPATH)
    de_novo_sequence = get_sequences("de_novo_sequence_p18.txt")[0]
    target_sequence = get_sequences("target_sequence_p18.txt")[0]
    alltrainingdata = get_sequences("training_sequences_p18.txt")
    metadata = "training_sequences_metadata.csv"

    # then change to the results directory for writing
    os.chdir(RESULTS_PATH)

    sequences_to_train_on = [1000]
    # sequences_to_train_on = [100, 500, 1000, 2000]

    trainfilename = "transformer_breakdown_breakdown_of_numtrain.csv"
    headers = ["Num Training Instances", "Full Accuracy", "Gap Accuracy", "Nongap Accuracy"]
    with open(trainfilename, "w+", encoding="utf-8", newline="") as f:
        train_writer = csv.writer(f)
        train_writer.writerow(headers)

    train_file = open(trainfilename, "a+", encoding="utf-8", newline="")
    train_writer = csv.writer(train_file)

    for numtrain in sequences_to_train_on:

        trainingdata = alltrainingdata[:numtrain]
        random.shuffle(trainingdata)

        incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices = np.where(de_novo_sequence == target_sequence)[0]

        fixer = ProteinScaffoldFixer(trainingdata, noise_percent=noise_percent, epochs=epochs,numgaps=numgaps)#today
        weights = fixer.train(verbose=1)

        hist = fixer.history.history
        training_loss = hist["loss"]
        validation_loss = hist["val_loss"]
        training_acc = hist["accuracy"]
        validation_acc = hist["val_accuracy"]

        headers = ["Training Accuracy", "Validation Accuracy", "Training Loss", "Validation Loss"]
        with open(f"transformer_results_on_original_{numtrain}.csv", "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(zip(training_acc, validation_acc, training_loss, validation_loss))

        predict_only_gaps = False
        pred = fixer.predict_sequence(de_novo_sequence, predict_only_gaps)

        incorrect_indices = np.where(target_sequence != pred)[0]
        correct_indices = np.where(target_sequence == pred)[0]
        indices_to_predict = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices_to_predict = np.intersect1d(correct_indices, indices_to_predict)

        gap_indices = np.where(de_novo_sequence == "-")[0]
        error_indices = np.setdiff1d(indices_to_predict, gap_indices)

        correct_gap_indices = np.intersect1d(gap_indices, correct_indices)
        correct_error_indices = np.intersect1d(error_indices, correct_indices)

        gap_acc = len(correct_gap_indices) / len(gap_indices)
        err_acc = len(correct_error_indices) / len(error_indices)
        full_acc = len(np.where(target_sequence == pred)[0]) / len(target_sequence)

        # print to console the prediction
        # Print to console
        print(f"Gap Acc for Transformer: {gap_acc}")
        print(f"Err Acc for Transformer: {err_acc}")
        print(f"Full Acc for Transformer: {full_acc}")
        print_sequence(pred, f"Predicted for {numtrain}", incorrect_indices, correct_indices_to_predict)

        # write the final accuracies according to the number of training instances
        with open(trainfilename, "a+", encoding="utf-8", newline="") as f:
            train_writer = csv.writer(f)
            train_writer.writerow([numtrain, full_acc, gap_acc, err_acc])

        # and save the predictions as images for use in the paper
        ut.write_protein_scaffold_image(pred, incorrect_indices, correct_indices_to_predict, f"transformer_predictions_on_original_{numtrain}")
    
    
    train_file.close()
    os.chdir(cwd)
    return weights
#==============================================================================
from termcolor import colored

def print_comparison_with_colors(string1, string2, width=80):
    """
    Prints two strings with a fixed width for each line, adding a space after each character,
    and comparing the characters at each position. If they are not the same, it colors the character
    from string1 in red and bold.

    :param string1: The first input string to be printed and compared.
    :param string2: The second input string to be compared with the first.
    :param width: The number of characters in each line including spaces.
    """
    # Remove any existing spaces from the strings
    string1 = string1.replace(' ', '')
    string2 = string2.replace(' ', '')
    
    # Add a space after each character in the strings
    spaced_string1 = ' '.join(string1[i] for i in range(len(string1)))
    spaced_string2 = ' '.join(string2[i] for i in range(len(string2)))

    # Initialize the index for the substring
    start_index = 0
    
    # The actual width needs to be adjusted because each character now takes two positions
    # (one for the character and one for the space)
    adjusted_width = width // 2

    # Loop through the strings, taking substrings of the specified width and comparing them
    while start_index < len(spaced_string1):
        # End index is the start index plus the width or the end of the string, whichever comes first
        end_index = min(start_index + width, len(spaced_string1))
        # Initialize a list to hold the formatted characters
        formatted_line = []
        # Compare the characters from string1 and string2
        for i in range(start_index, end_index, 2):  # Step by 2 to skip spaces
            if i < len(spaced_string2) and spaced_string1[i] != spaced_string2[i]:
                # If characters don't match, append the red bold character from string1
                formatted_line.append(colored(spaced_string1[i], 'red', attrs=['bold']))
            else:
                # Otherwise, append the character from string1 as it is
                formatted_line.append(spaced_string1[i])
            # Append a space after each character
            formatted_line.append(' ')
        
        # Join the formatted characters and print the line
        print(''.join(formatted_line))
        # Increment the start_index to move to the next substring
        start_index += width
# =============================================================================


In [4]:
# Input strings provided by the user
target_seq_ = ('DIQMTQSPSSLSASVGDRVTITCQASRDISNYLNWYQQKPGKAPMLLIYAASNLQTGVPSRFSGSGSG\
TDFTFTISSLQPEDLATYYCQQYGNLPLTFGGGTKVEIKGTVAAPSVFIFPPSDEQLKSGTASVVCLL\
NNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQG\
LSSPVTKSFNRGEC')

de_novo_seq_ = (
    '--QMTQ-PSSLS-SVGDRVF---QASR-ISN-L-W-QQKP--APMLLIYAASN-QTGVP-RFSGS---\
TDFTFTISSLQ-E-LATYYCQQYGN--L--GGGTK-E-KGTTA-PIVFIFP-SDE-LKSM---VVCLLNNFYP-EAKV--KVDG\
-L-SGNS-ESVT--D-NDS--SLS-TL-LS----EK--V-ECEVTHQG---PVTKSFLR-EC')

In [5]:
test_original_protein_scaffold()

2024-01-11 08:50:37.355104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2024-01-11 08:50:37.355780: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14171 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-01-11 08:50:37.356221: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14171 MB memory:  -> device: 2, name: Quadro RTX 5000, pci bus id: 0000:67:00.0, compute capability: 7.5
2024-01-11 08:50:37.356650: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13515 MB memory:  -> device: 3, name: Quadro RTX 5000, pci bus id: 0000:68:00.0, comp

Input shape: (740, 24)
Epoch 1/10


2024-01-11 08:50:45.021001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


219/219 [==============================] - 137s 579ms/step - loss: 0.5392 - accuracy: 0.8425 - val_loss: 0.2001 - val_accuracy: 0.9383
Epoch 2/10
219/219 [==============================] - 126s 577ms/step - loss: 0.1162 - accuracy: 0.9673 - val_loss: 0.0680 - val_accuracy: 0.9801
Epoch 3/10
219/219 [==============================] - 126s 577ms/step - loss: 0.0768 - accuracy: 0.9781 - val_loss: 0.0536 - val_accuracy: 0.9841
Epoch 4/10
219/219 [==============================] - 126s 577ms/step - loss: 0.0660 - accuracy: 0.9809 - val_loss: 0.0472 - val_accuracy: 0.9855
Epoch 5/10
219/219 [==============================] - 126s 577ms/step - loss: 0.0599 - accuracy: 0.9823 - val_loss: 0.0424 - val_accuracy: 0.9868
Epoch 6/10
219/219 [==============================] - 126s 577ms/step - loss: 0.0563 - accuracy: 0.9833 - val_loss: 0.0397 - val_accuracy: 0.9873
Epoch 7/10
219/219 [==============================] - 126s 577ms/step - loss: 0.0533 - accuracy: 0.9839 - val_loss: 0.0369 - val_accura

In [6]:
print_comparison_with_colors(de_novo_seq_, target_seq_)

- - Q M T Q - P S S L S - S V G D R V F - - - Q A S R - I S N - L - W - Q Q K P 
- - A P M L L I Y A A S N - Q T G V P - R F S G S - - - T D F T F T I S S L Q - 
E - L A T Y Y C Q Q Y G N - - L - - G G G T K - E - K G T T A - P I V F I F P - 
S D E - L K S M - - - V V C L L N N F Y P - E A K V - - K V D G - L - S G N S - 
E S V T - - D - N D S - - S L S - T L - L S - - - - E K - - V - E C E V T H Q G 
- - - P V T K S F L R - E C 


### Transformer Protein Scaffold Filling - mab

In [7]:
DATAPATH = os.path.join(os.getcwd(), "newdatasets_mab")
RESULTS_PATH = os.path.join(os.getcwd(), "transformer_results-mab")

In [12]:
'''
Trial summary
Hyperparameters:
conv1_filter_size: 10
conv2_filter_size: 9
conv1_filters: 50
conv2_filters: 66
bridge_filters: 120
max_pool: 2
dropout_: : 0.35
Score: 0.9727120995521545


Hyperparameters:
conv1_filter_size: 5
conv2_filter_size: 9
conv1_filters: 46
conv2_filters: 90
bridge_filters: 160
bridge_filter_size: 5
max_pool: 2
dropout_: : 0.5
Score: 0.9742878675460815
'''

# # =============================================================================
# class Encoder(layers.Layer):
#     """Encoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=10, conv2_filter_size=9, maxpool=2, dropout=0.25, name="encoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1D(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1D(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.maxpool = layers.MaxPooling1D(maxpool, padding="same")
#         self.dropout = layers.Dropout(dropout)
        

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.maxpool(self.conv2(self.dropout(self.maxpool(self.conv1(inputs))))))

# # =============================================================================
# class Decoder(layers.Layer):
#     """Decoder part of autoencoder"""

#     # -------------------------------------------------------------------------
#     def __init__(self, conv1_filters, conv2_filters, conv1_filter_size=5, conv2_filter_size=9, maxpool=2, dropout=0.25, name="decoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.conv1 = layers.Conv1DTranspose(conv1_filters, conv1_filter_size, padding="same", activation="relu")
#         self.conv2 = layers.Conv1DTranspose(conv2_filters, conv2_filter_size, padding="same", activation="relu")
#         self.upsample = layers.UpSampling1D(maxpool)
#         self.dropout = layers.Dropout(dropout)

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.dropout(self.upsample(self.conv2(self.dropout(self.upsample(self.conv1(inputs))))))

# # =============================================================================
# class Autoencoder(Model):
#     """Autoencoder"""
#     # -------------------------------------------------------------------------
#     def __init__(self, num_classes, name="autoencoder", **kwargs):
#         super().__init__(name=name, **kwargs)

#         self.num_classes = num_classes
#         self.hyperparameters = {
#             "conv1_filters": 46,
#             "conv2_filters": 90,
#             "conv1_filter_size": 5,
#             "conv2_filter_size": 9,
#             "bridge_filters": 160,
#             "bridge_filter_size": 5,
#             "dropout": 0.50,
#             "maxpool": 5,
#         }

#         hp = self.hyperparameters

#         self.encoder = Encoder(hp["conv1_filters"], hp["conv2_filters"])
#         self.bridge = layers.Conv1D(hp["bridge_filters"], hp["bridge_filter_size"], padding="same", activation="relu")
#         self.decoder = Decoder(hp["conv2_filters"], hp["conv1_filters"])
#         self.finallayer = layers.Conv1D(self.num_classes, hp["conv1_filter_size"], padding="same", activation="softmax")

#     # -------------------------------------------------------------------------
#     def call(self, inputs):
#         return self.finallayer(self.decoder(self.bridge(self.encoder(inputs))))

    
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class Transformer(Model):
    def __init__(self, input_shape_, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, 
                 mlp_dropout=0,name="transformers", **kwargs):
        super().__init__(name=name, **kwargs)
        self.input_shape_ = input_shape_
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout

    def transformer_encoder(self, inputs):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
        )(x, x)
        x = layers.Dropout(self.dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(self.dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    def build_model(self):
        print("Input shape:", self.input_shape_)
        inputs = tf.keras.Input(shape=self.input_shape_)
        x = inputs
        for _ in range(self.num_transformer_blocks):
            x = self.transformer_encoder(x)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in self.mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(self.mlp_dropout)(x)
        outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
        outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
        return models.Model(inputs, outputs)

#     def call(self, inputs):
#         x = inputs
#         for _ in range(self.num_transformer_blocks):
#             x = self.transformer_encoder(x)

#         x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
#         for dim in self.mlp_units:
#             x = layers.Dense(dim, activation="relu")(x)
#             x = layers.Dropout(self.mlp_dropout)(x)
#         outputs = layers.Dense(np.prod(self.input_shape_), activation="softmax")(x)
#         outputs = layers.Reshape(target_shape=self.input_shape_, name='out_recon')(outputs)
#         return outputs
        
# =============================================================================
class PaddingType(Enum):
    ZERO=1
    TRUNCATE=2
    EMPTY=3

# =============================================================================

class NoisificationMethod(Enum):
    RANDOMSCATTER=1
    RANDOMCONTIG=2

weights = None

# =============================================================================
class ProteinScaffoldFixer():
    """Class to correct errors in a protein scaffold and fill gaps"""

    # -------------------------------------------------------------------------
    def __init__(self, 
                output_seqs, 
                paddingtype=PaddingType.EMPTY, 
                noise_percent=np.array([0.10,0.09,0.08,0.07,0.06]),#0.2, 
                noisemethod=NoisificationMethod.RANDOMSCATTER, 
                numgaps=np.array([4, 6, 8, 10]),#5, 
                mingapsize=3, 
                mincontigsize=1, 
                epochs=250,
                optimizer="adam",
                early_stopping=False):

        self.epochs = epochs
        self.numgaps = numgaps#added today
        self.optimizer = optimizer
        self.paddingtype = paddingtype
        self.noise_percent = noise_percent
        self.early_stopping = early_stopping

        self.classes = np.array(["-", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z'])
        self.ohe = OneHotEncoder(sparse_output=False, categories=[self.classes])

        self.max_seq_length = max((len(seq) for seq in output_seqs))
        self.maxpool = 2
        while self.max_seq_length % (self.maxpool * self.maxpool) != 0:
            self.max_seq_length += 1

        self.output_seqs = output_seqs
        #self.input_seqs = self.noisify_sequences(output_seqs, noise_percent, noisemethod, numgaps, mingapsize, mincontigsize)
        #########################
        self.output_seqs_all = []
        self.input_seqs_all = []
        
        for n_gaps in numgaps:
            for p_missing in noise_percent:
                self.output_seqs_temp = output_seqs
                self.input_seqs_temp = self.noisify_sequences(self.output_seqs_temp, p_missing, noisemethod, n_gaps, mingapsize=3, mincontigsize=1)
                self.output_seqs_all += self.output_seqs_temp
                self.input_seqs_all += self.input_seqs_temp
        #########################
        self.train_x = self.preprocess_sequences(self.input_seqs_all)
        self.train_y = self.preprocess_sequences(self.output_seqs_all)

        self.history = None

        #self.autoencoder = Autoencoder(len(self.classes))
        ### my additions ######
        self.input_shape = self.train_x.shape[1:]#(440,24)
        self.head_size=256
        self.num_heads=4
        self.ff_dim=4
        self.num_transformer_blocks=4#3
        self.mlp_units=[512,128]
        self.mlp_dropout=0.4
        self.dropout=0.25
        self.transformer = Transformer(self.input_shape, self.head_size, self.num_heads, self.ff_dim, 
                                self.num_transformer_blocks, self.mlp_units, self.dropout, self.mlp_dropout)
        #self.transformer = self.transformer.build_model()
    # =============================================================================
    def noisify_sequences(self, seqs, noise_percent, noisemethod, numgaps=5, mingapsize=3, mincontigsize=3):
        if noisemethod == NoisificationMethod.RANDOMSCATTER:
            return self.noisify_by_random_scatter(seqs, noise_percent)
        elif noisemethod == NoisificationMethod.RANDOMCONTIG:
            return self.noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize)

    # =============================================================================
    def noisify_by_random_scatter(self, seqs, noise_percent):
        sequences = deepcopy(seqs)
        # To noisify our input data, we will replace random amino acids with something else
        for seq in sequences:

            # We randomly sample from all the possible indices of seq
            indices_to_replace = random.sample(range(len(seq)), int(noise_percent * len(seq)))
            seq[indices_to_replace] = "-"

        return sequences

    # =============================================================================
    def noisify_by_random_contigs(self, seqs, noise_percent, numgaps, mingapsize, mincontigsize):

        sequences = deepcopy(seqs)
        for seqind, seq in enumerate(sequences):

            amino_acid_length = len(seq)
            amino_acids_to_replace = int(amino_acid_length * noise_percent)

            # The idea is to build a gap queue, randomly putting each amino acid to replace into each gap
            # Then, we build a contig queue, randomly putting each amino acid into each contig
            gap_queue = np.zeros(numgaps).astype(int)
            for _ in range(amino_acids_to_replace):

                # Any gap can be considered
                valid_gaps = np.arange(numgaps)

                # ... so long as we have no underfilled gaps
                underfilled = gap_queue < mingapsize
                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_gaps = valid_gaps[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                gap_queue[np.random.choice(valid_gaps)] += 1

            # There can always be one more contig than gaps (if there's a contig at beginnning and end of sequence)
            contig_queue = np.zeros(numgaps+1).astype(int)

            # We have to allocate all the amino acids NOT in gaps into contigs BETWEEN the gaps
            for _ in range(amino_acid_length - amino_acids_to_replace):

                # Any contig can be considered
                valid_contigs = np.arange(len(contig_queue))

                # ... so long as we have no underfilled contigs
                underfilled = contig_queue < mincontigsize

                # The exceptions are the first and last contigs. They are never considered underfilled
                underfilled[0] = False
                underfilled[-1] = False

                # If we have any underfilled, then consider only those until they are no longer underfilled
                if np.any(underfilled):
                    valid_contigs = valid_contigs[np.where(underfilled)]
                
                # Once valid gap indices have been determined, randomly pick one to increment
                contig_queue[np.random.choice(valid_contigs)] += 1
            
            # Once we have determined gap_queue and contig_queue, we iterate over them to set the gaps equal
            # to our blank amino acid character

            sequence_pointer = 0
            iscontig=True
            while sequence_pointer < len(seq):
                if iscontig:
                    # Don't do anything for the contig except increment the pointer and pop off the contig queue
                    sequence_pointer += contig_queue[0]
                    contig_queue = np.delete(contig_queue, 0)
                else:
                    seq[sequence_pointer:sequence_pointer+gap_queue[0]] = '-'
                    sequence_pointer += gap_queue[0]
                    gap_queue = np.delete(gap_queue, 0)

                # We alternate between contigs and gaps
                iscontig = not iscontig

        return sequences

    # -------------------------------------------------------------------------
    def predict_sequence(self, seq, predict_only_gaps):

        scaffold = self.preprocess_sequences([seq])
        #pred = self.autoencoder.predict(scaffold).reshape(self.max_seq_length, len(self.classes))
        pred = self.transformer.predict(scaffold).reshape(self.max_seq_length, len(self.classes))

        # Set the probability of empty "-" to zero, since we always want to predict something
        emptyclass = np.where(self.ohe.transform(np.array("-").reshape(-1, 1))[0])[0][0]
        pred[:, emptyclass] = 0.0

        # Convert the probability distribution to a one-hot encoded vector
        mask = pred == np.amax(pred, axis=1).reshape(pred.shape[0], 1)
        indices = list((i, np.where(mask[i])[0][0]) for i in range(mask.shape[0]))

        pred = np.zeros(pred.shape)
        for i in indices:
            pred[i] = 1

        # Then we can use our one hot encoder to convert back to the original sequence of classes
        pred = self.ohe.inverse_transform(pred[:len(seq), :len(self.classes)]).reshape(len(seq))

        if predict_only_gaps:
            # We only care about predicting the gaps in seq, so replace amino acids in prediction with original nongaps
            nongaps = np.where(seq != '-')[0]
            pred[nongaps] = seq[nongaps]

        return pred

    # =============================================================================
    def preprocess_sequences(self, seqs: List[np.array]) -> np.array:

        # the value -1 lets numpy know to infer the shape. So it's just a column vector of length num_samples
        seqs = [np.array(seq).reshape(-1, 1) for seq in seqs]

        # One-hot encode each sequence
        seqs = [self.ohe.fit_transform(seq) for seq in seqs]

        if self.paddingtype == PaddingType.ZERO:
            # The sequences may have different lengths, so we will pad them with zeros
            # We are padding to fill up to max length, then we can turn it into a single numpy tensor
            return np.array([np.pad(seq, ((0, self.max_seq_length-len(seq)), (0, 0))) for seq in seqs])

        elif self.paddingtype == PaddingType.EMPTY:

            # We pad with the "empty" class
            emptyclass = self.ohe.fit_transform(np.array("-").reshape(-1, 1))[0]

            # dynamically extend each seq by enough emptyvals to make a single sequence length
            return np.array([np.vstack((seq, *(emptyclass for _ in range(self.max_seq_length-len(seq))))) for seq in seqs])

        elif self.paddingtype == PaddingType.TRUNCATE:
            # FIXME: Implement me
            raise Exception("Have not implemented Padding Type TRUNCATE!")

    # =============================================================================
    def train(self, verbose="auto", optimizer="adam", epochs=None):
        self.optimizer = optimizer
        if epochs:
            self.epochs = epochs

#         self.autoencoder.compile(optimizer=self.optimizer, loss="categorical_crossentropy",
#                                  metrics=["accuracy"])
        self.transformer = self.transformer.build_model()
        self.transformer.compile(loss="categorical_crossentropy",
                                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                                    metrics=["accuracy"],)
        callbacks = []
        if self.early_stopping:
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10))
        #self.history = self.autoencoder.fit(self.train_x, self.train_y, epochs=self.epochs, validation_split=0.15, verbose=verbose, callbacks=callbacks)
        self.history = self.transformer.fit(self.train_x, self.train_y, epochs=self.epochs, batch_size=64,
                                            validation_split=0.30, verbose=verbose, callbacks=callbacks)
        # Save the history to a file using pickle
        with open('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-mab/model_p5A/model_history.pkl', 'wb') as file:
            pk.dump(self.history, file)
        #plot_model(self.transformer, to_file='transformer_plot.png', show_shapes=True, show_layer_names=True)
        #print(self.transformer.summary())
        self.transformer.save('/home/rkannan/Desktop/PSF_Transformers/Good Results backup/cda_for_protein_scaffold_filling-main (Transformers) (all)/transformer_results-mab/model_p5A/transormer_model_p5A.h5')
        
# =============================================================================
def highlight_indices(seq: np.array, indices: np.array, color: str):
    # We use deepcopy to prevent mutation and we cast to object so that we can treat contents as python strings
    # otherwise, it gets messed up as it treats each element as a single character
    newseq = deepcopy(seq).astype('object')
    newseq[indices] = np.vectorize(lambda x: colored(x, color, attrs=["bold"]))(seq[indices])
    return newseq

# =============================================================================
def print_sequence(seq, 
                   header: str=None, 
                   incorrect_indices: Optional[np.array]=None, 
                   correct_indices: Optional[np.array]=None):

    newseq = deepcopy(seq)
    if correct_indices is not None and correct_indices.size != 0:
        newseq = highlight_indices(newseq, correct_indices, "green")
    if incorrect_indices is not None and incorrect_indices.size != 0:
        newseq = highlight_indices(newseq, incorrect_indices, "red")

    line_length = 40
    if header:
        print(header)
    print("=" * line_length)

    i = 0
    while i < len(newseq):
        print(" ".join(newseq[i: i+line_length]))
        i += line_length

# =============================================================================
def get_sequences(fasta_file: str) -> List[np.array]:
    sequences = []
    lines = []
    with open(fasta_file, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append(np.array([c for c in "".join(parts)]))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append(np.array([c for c in "".join(parts)]))
    return sequences

# =============================================================================
def snake_case_prettify(s):
    return " ".join(w.capitalize() for w in s.split("_"))

# =============================================================================
def save_models(models):
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if not os.path.exists(modeldir):
        os.makedirs(modeldir)

    for name, model in models.items():
        model.save(os.path.join(modeldir, name))

# =============================================================================
def load_models():
    modeldir = os.path.join(os.getcwd(), "1Dmodels")
    if os.path.exists(modeldir):
        return {f: tf.keras.models.load_model(os.path.join(modeldir, f)) for f in os.listdir(modeldir)}
    return {}

# =============================================================================
def print_diff_between_target_and_de_novo():

    dataset_dir = os.path.join(os.getcwd(), "newdatasets_mab")
    
    de_novo_sequence = get_sequences(os.path.join(dataset_dir, "de_novo_sequence_mab.txt"))[0]
    target_sequence = get_sequences(os.path.join(dataset_dir, "target_sequence_mab.txt"))[0]

    gap_indices = np.where(de_novo_sequence == '-')[0]
    incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
    correct_indices = np.where(de_novo_sequence == target_sequence)[0]
    incorrect_non_gaps = np.setdiff1d(incorrect_indices, gap_indices)
    print(f"Length of target: {len(target_sequence)}")
    print(f"Number of incorrect non-gaps: {len(incorrect_non_gaps)}")
    print(f"Number of gaps: {len(gap_indices)}")

    print_sequence(de_novo_sequence, "Protein Scaffold", incorrect_indices, correct_indices)

# =============================================================================
def test_original_protein_scaffold():

    ut.mkdir_if_not_exists(RESULTS_PATH)
    ut.mkdir_if_not_exists(DATAPATH)

    epochs = 10
    percent_missings = np.array([0.10,0.09,0.08,0.07,0.06])
    num_gaps = np.array([4, 6, 8, 10])
    noise_percent = np.array([0.10,0.09,0.08,0.07,0.06])#0.25
    numgaps = np.array([4, 6, 8, 10])#5#just added today

    cwd = os.getcwd()

    # change to the location of the data for reading
    os.chdir(DATAPATH)
    de_novo_sequence = get_sequences("de_novo_sequence_mab.txt")[0]
    target_sequence = get_sequences("target_sequence_mab.txt")[0]
    alltrainingdata = get_sequences("training_sequences_mab.txt")
    metadata = "training_sequences_metadata.csv"

    # then change to the results directory for writing
    os.chdir(RESULTS_PATH)

    sequences_to_train_on = [1000]
    # sequences_to_train_on = [100, 500, 1000, 2000]

    trainfilename = "transformer_breakdown_breakdown_of_numtrain.csv"
    headers = ["Num Training Instances", "Full Accuracy", "Gap Accuracy", "Nongap Accuracy"]
    with open(trainfilename, "w+", encoding="utf-8", newline="") as f:
        train_writer = csv.writer(f)
        train_writer.writerow(headers)

    train_file = open(trainfilename, "a+", encoding="utf-8", newline="")
    train_writer = csv.writer(train_file)

    for numtrain in sequences_to_train_on:

        trainingdata = alltrainingdata[:numtrain]
        random.shuffle(trainingdata)

        incorrect_indices = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices = np.where(de_novo_sequence == target_sequence)[0]

        fixer = ProteinScaffoldFixer(trainingdata, noise_percent=noise_percent, epochs=epochs,numgaps=numgaps)#today
        weights = fixer.train(verbose=1)

        hist = fixer.history.history
        training_loss = hist["loss"]
        validation_loss = hist["val_loss"]
        training_acc = hist["accuracy"]
        validation_acc = hist["val_accuracy"]

        headers = ["Training Accuracy", "Validation Accuracy", "Training Loss", "Validation Loss"]
        with open(f"transformer_results_on_original_{numtrain}.csv", "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(zip(training_acc, validation_acc, training_loss, validation_loss))

        predict_only_gaps = False
        pred = fixer.predict_sequence(de_novo_sequence, predict_only_gaps)

        incorrect_indices = np.where(target_sequence != pred)[0]
        correct_indices = np.where(target_sequence == pred)[0]
        indices_to_predict = np.where(de_novo_sequence != target_sequence)[0]
        correct_indices_to_predict = np.intersect1d(correct_indices, indices_to_predict)

        gap_indices = np.where(de_novo_sequence == "-")[0]
        error_indices = np.setdiff1d(indices_to_predict, gap_indices)

        correct_gap_indices = np.intersect1d(gap_indices, correct_indices)
        correct_error_indices = np.intersect1d(error_indices, correct_indices)

        gap_acc = len(correct_gap_indices) / len(gap_indices)
        err_acc = len(correct_error_indices) / len(error_indices)
        full_acc = len(np.where(target_sequence == pred)[0]) / len(target_sequence)

        # print to console the prediction
        # Print to console
        print(f"Gap Acc for Transformer: {gap_acc}")
        print(f"Err Acc for Transformer: {err_acc}")
        print(f"Full Acc for Transformer: {full_acc}")
        print_sequence(pred, f"Predicted for {numtrain}", incorrect_indices, correct_indices_to_predict)

        # write the final accuracies according to the number of training instances
        with open(trainfilename, "a+", encoding="utf-8", newline="") as f:
            train_writer = csv.writer(f)
            train_writer.writerow([numtrain, full_acc, gap_acc, err_acc])

        # and save the predictions as images for use in the paper
        ut.write_protein_scaffold_image(pred, incorrect_indices, correct_indices_to_predict, f"transformer_predictions_on_original_{numtrain}")
    
    
    train_file.close()
    os.chdir(cwd)
    return weights
#==============================================================================
from termcolor import colored

def print_comparison_with_colors(string1, string2, width=80):
    """
    Prints two strings with a fixed width for each line, adding a space after each character,
    and comparing the characters at each position. If they are not the same, it colors the character
    from string1 in red and bold.

    :param string1: The first input string to be printed and compared.
    :param string2: The second input string to be compared with the first.
    :param width: The number of characters in each line including spaces.
    """
    # Remove any existing spaces from the strings
    string1 = string1.replace(' ', '')
    string2 = string2.replace(' ', '')
    
    # Add a space after each character in the strings
    spaced_string1 = ' '.join(string1[i] for i in range(len(string1)))
    spaced_string2 = ' '.join(string2[i] for i in range(len(string2)))

    # Initialize the index for the substring
    start_index = 0
    
    # The actual width needs to be adjusted because each character now takes two positions
    # (one for the character and one for the space)
    adjusted_width = width // 2

    # Loop through the strings, taking substrings of the specified width and comparing them
    while start_index < len(spaced_string1):
        # End index is the start index plus the width or the end of the string, whichever comes first
        end_index = min(start_index + width, len(spaced_string1))
        # Initialize a list to hold the formatted characters
        formatted_line = []
        # Compare the characters from string1 and string2
        for i in range(start_index, end_index, 2):  # Step by 2 to skip spaces
            if i < len(spaced_string2) and spaced_string1[i] != spaced_string2[i]:
                # If characters don't match, append the red bold character from string1
                formatted_line.append(colored(spaced_string1[i], 'red', attrs=['bold']))
            else:
                # Otherwise, append the character from string1 as it is
                formatted_line.append(spaced_string1[i])
            # Append a space after each character
            formatted_line.append(' ')
        
        # Join the formatted characters and print the line
        print(''.join(formatted_line))
        # Increment the start_index to move to the next substring
        start_index += width
# =============================================================================


In [13]:
# Input strings provided by the user
target_seq_ = ('DIQMTQSPSSLSASVGDRVTITCKASQNIDKYLNWYQQKPGKAPKLLIYNTNNLQTGVPSRFSGSGSGTD\
FTFTISSLQPEDIATYYCLQHISRPRTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQS\
GNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC')

de_novo_seq_ = (
    '---MTQSPSSISASVGDRVTITCK---NIDKYINWYQQKPGKAPKIIIYNTNNIQTGVPSRF---G----FTFTI-----------\
    YCIQHISRPRTFGQGTKVEIKRSIAAPSVFIFPPSDEQIKSGTASVVCIINNFYPREAQPRRKVDNAIQSGNSQESVTEQDSKDSTYS\
    ISSTITISKADYEKHKVYACEVTHQGISSPVTKSFN----')

In [14]:
test_original_protein_scaffold()

Input shape: (452, 24)
Epoch 1/10
219/219 [==============================] - 67s 270ms/step - loss: 0.5839 - accuracy: 0.8305 - val_loss: 0.1442 - val_accuracy: 0.9586
Epoch 2/10
219/219 [==============================] - 59s 268ms/step - loss: 0.1416 - accuracy: 0.9594 - val_loss: 0.0918 - val_accuracy: 0.9734
Epoch 3/10
219/219 [==============================] - 59s 268ms/step - loss: 0.1099 - accuracy: 0.9682 - val_loss: 0.0761 - val_accuracy: 0.9768
Epoch 4/10
219/219 [==============================] - 59s 268ms/step - loss: 0.0979 - accuracy: 0.9712 - val_loss: 0.0671 - val_accuracy: 0.9784
Epoch 5/10
219/219 [==============================] - 59s 269ms/step - loss: 0.0901 - accuracy: 0.9727 - val_loss: 0.0608 - val_accuracy: 0.9803
Epoch 6/10
219/219 [==============================] - 59s 269ms/step - loss: 0.0845 - accuracy: 0.9740 - val_loss: 0.0573 - val_accuracy: 0.9814
Epoch 7/10
219/219 [==============================] - 59s 269ms/step - loss: 0.0813 - accuracy: 0.9746 - va

In [15]:
print_comparison_with_colors(de_novo_seq_, target_seq_)

- - - M T Q S P S S I S A S V G D R V T I T C K - - - N I D K Y I N W Y Q Q K P 
G K A P K I I I Y N T N N I Q T G V P S R F - - - G - - - - F T F T I - - - - - 
- - - - - - Y C I Q H I S R P R T F G Q G T K V E I K R S I A A P S V F I F P P 
S D E Q I K S G T A S V V C I I N N F Y P R E A Q P R R K V D N A I Q S G N S Q 
E S V T E Q D S K D S T Y S I S S T I T I S K A D Y E K H K V Y A C E V T H Q G 
I S S P V T K S F N - - - - 


In [9]:
test_generated_datasets()

Input shape: (440, 24)
Epoch 1/200
27/27 [==============================] - 6s 135ms/step - loss: 1.3725 - accuracy: 0.6198 - val_loss: 1.1466 - val_accuracy: 0.7096
Epoch 2/200
27/27 [==============================] - 3s 120ms/step - loss: 0.8897 - accuracy: 0.7314 - val_loss: 1.0648 - val_accuracy: 0.7224
Epoch 3/200
27/27 [==============================] - 3s 120ms/step - loss: 0.8683 - accuracy: 0.7354 - val_loss: 1.0730 - val_accuracy: 0.7225
Epoch 4/200
27/27 [==============================] - 3s 119ms/step - loss: 0.8522 - accuracy: 0.7370 - val_loss: 0.9652 - val_accuracy: 0.7234
Epoch 5/200
27/27 [==============================] - 3s 120ms/step - loss: 0.6484 - accuracy: 0.7906 - val_loss: 0.7742 - val_accuracy: 0.8025
Epoch 6/200
27/27 [==============================] - 3s 121ms/step - loss: 0.3762 - accuracy: 0.8917 - val_loss: 0.5816 - val_accuracy: 0.8710
Epoch 7/200
27/27 [==============================] - 3s 121ms/step - loss: 0.2648 - accuracy: 0.9279 - val_loss: 0.5031

 input_8 (InputLayer)           [(None, 440, 24)]    0           []                               
                                                                                                  
 layer_normalization_54 (LayerN  (None, 440, 24)     48          ['input_8[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_27 (Multi  (None, 440, 24)     101400      ['layer_normalization_54[0][0]', 
 HeadAttention)                                                   'layer_normalization_54[0][0]'] 
                                                                                                  
 dropout_69 (Dropout)           (None, 440, 24)      0           ['multi_head_attention_27[0][0]']
                                                                                                  
 tf.__oper

 ormalization)                                                                                    
                                                                                                  
 conv1d_60 (Conv1D)             (None, 440, 4)       100         ['layer_normalization_61[0][0]'] 
                                                                                                  
 dropout_76 (Dropout)           (None, 440, 4)       0           ['conv1d_60[0][0]']              
                                                                                                  
 conv1d_61 (Conv1D)             (None, 440, 24)      120         ['dropout_76[0][0]']             
                                                                                                  
 tf.__operators__.add_61 (TFOpL  (None, 440, 24)     0           ['conv1d_61[0][0]',              
 ambda)                                                           'tf.__operators__.add_60[0][0]']
          

27/27 [==============================] - 3s 121ms/step - loss: 0.1083 - accuracy: 0.9676 - val_loss: 0.4373 - val_accuracy: 0.9086
Epoch 29/200
27/27 [==============================] - 3s 121ms/step - loss: 0.1084 - accuracy: 0.9671 - val_loss: 0.4189 - val_accuracy: 0.9114
Epoch 30/200
27/27 [==============================] - 3s 121ms/step - loss: 0.1085 - accuracy: 0.9672 - val_loss: 0.4313 - val_accuracy: 0.9115
Epoch 31/200
27/27 [==============================] - 3s 121ms/step - loss: 0.1040 - accuracy: 0.9683 - val_loss: 0.4189 - val_accuracy: 0.9116
Epoch 32/200
27/27 [==============================] - 3s 122ms/step - loss: 0.1051 - accuracy: 0.9681 - val_loss: 0.4289 - val_accuracy: 0.9108
Epoch 33/200
27/27 [==============================] - 3s 121ms/step - loss: 0.1017 - accuracy: 0.9687 - val_loss: 0.4441 - val_accuracy: 0.9098
Epoch 34/200
27/27 [==============================] - 3s 121ms/step - loss: 0.1037 - accuracy: 0.9682 - val_loss: 0.4308 - val_accuracy: 0.9091
Epoch

                                                                                                  
 multi_head_attention_33 (Multi  (None, 440, 24)     101400      ['layer_normalization_66[0][0]', 
 HeadAttention)                                                   'layer_normalization_66[0][0]'] 
                                                                                                  
 dropout_83 (Dropout)           (None, 440, 24)      0           ['multi_head_attention_33[0][0]']
                                                                                                  
 tf.__operators__.add_66 (TFOpL  (None, 440, 24)     0           ['dropout_83[0][0]',             
 ambda)                                                           'tf.__operators__.add_65[0][0]']
                                                                                                  
 layer_normalization_67 (LayerN  (None, 440, 24)     48          ['tf.__operators__.add_66[0][0]']
 ormalizat

...

 conv1d_131 (Conv1D)            (None, 284, 24)      120         ['dropout_164[0][0]']            
                                                                                                  
 tf.__operators__.add_131 (TFOp  (None, 284, 24)     0           ['conv1d_131[0][0]',             
 Lambda)                                                          'tf.__operators__.add_130[0][0]'
                                                                 ]                                
                                                                                                  
 layer_normalization_132 (Layer  (None, 284, 24)     48          ['tf.__operators__.add_131[0][0]'
 Normalization)                                                  ]                                
                                                                                                  
 multi_head_attention_66 (Multi  (None, 284, 24)     101400      ['layer_normalization_132[0][0]',
 HeadAtten

Epoch 8/200
27/27 [==============================] - 2s 72ms/step - loss: 0.3891 - accuracy: 0.8858 - val_loss: 0.5364 - val_accuracy: 0.8468
Epoch 9/200
27/27 [==============================] - 2s 72ms/step - loss: 0.3159 - accuracy: 0.9077 - val_loss: 0.4704 - val_accuracy: 0.8723
Epoch 10/200
27/27 [==============================] - 2s 72ms/step - loss: 0.2732 - accuracy: 0.9228 - val_loss: 0.4383 - val_accuracy: 0.8872
Epoch 11/200
27/27 [==============================] - 2s 72ms/step - loss: 0.2419 - accuracy: 0.9331 - val_loss: 0.4055 - val_accuracy: 0.9014
Epoch 12/200
27/27 [==============================] - 2s 71ms/step - loss: 0.2169 - accuracy: 0.9396 - val_loss: 0.3912 - val_accuracy: 0.9052
Epoch 13/200
27/27 [==============================] - 2s 72ms/step - loss: 0.1982 - accuracy: 0.9459 - val_loss: 0.3666 - val_accuracy: 0.9118
Epoch 14/200
27/27 [==============================] - 2s 72ms/step - loss: 0.1826 - accuracy: 0.9492 - val_loss: 0.3539 - val_accuracy: 0.9193
E

 multi_head_attention_67 (Multi  (None, 296, 24)     101400      ['layer_normalization_134[0][0]',
 HeadAttention)                                                   'layer_normalization_134[0][0]']
                                                                                                  
 dropout_169 (Dropout)          (None, 296, 24)      0           ['multi_head_attention_67[0][0]']
                                                                                                  
 tf.__operators__.add_134 (TFOp  (None, 296, 24)     0           ['dropout_169[0][0]',            
 Lambda)                                                          'input_18[0][0]']               
                                                                                                  
 layer_normalization_135 (Layer  (None, 296, 24)     48          ['tf.__operators__.add_134[0][0]'
 Normalization)                                                  ]                                
          

 layer_normalization_141 (Layer  (None, 296, 24)     48          ['tf.__operators__.add_140[0][0]'
 Normalization)                                                  ]                                
                                                                                                  
 conv1d_140 (Conv1D)            (None, 296, 4)       100         ['layer_normalization_141[0][0]']
                                                                                                  
 dropout_176 (Dropout)          (None, 296, 4)       0           ['conv1d_140[0][0]']             
                                                                                                  
 conv1d_141 (Conv1D)            (None, 296, 24)      120         ['dropout_176[0][0]']            
                                                                                                  
 tf.__operators__.add_141 (TFOp  (None, 296, 24)     0           ['conv1d_141[0][0]',             
 Lambda)  

                                                                                                  
 dropout_182 (Dropout)          (None, 440, 4)       0           ['conv1d_144[0][0]']             
                                                                                                  
 conv1d_145 (Conv1D)            (None, 440, 24)      120         ['dropout_182[0][0]']            
                                                                                                  
 tf.__operators__.add_145 (TFOp  (None, 440, 24)     0           ['conv1d_145[0][0]',             
 Lambda)                                                          'tf.__operators__.add_144[0][0]'
                                                                 ]                                
                                                                                                  
 layer_normalization_146 (Layer  (None, 440, 24)     48          ['tf.__operators__.add_145[0][0]'
 Normaliza

In [5]:
def noisify_sequences(seqs, noise_percent, noisemethod, numgaps=5, mingapsize=3, mincontigsize=3):
    if noisemethod == NoisificationMethod.RANDOMSCATTER:
        return noisify_by_random_scatter(seqs, noise_percent)
    elif noisemethod == NoisificationMethod.RANDOMCONTIG:
        return noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize)

# =============================================================================
def noisify_by_random_scatter(seqs, noise_percent):
    sequences = deepcopy(seqs)
    # To noisify our input data, we will replace random amino acids with something else
    for seq in sequences:

        # We randomly sample from all the possible indices of seq
        indices_to_replace = random.sample(range(len(seq)), int(noise_percent * len(seq)))
        seq[indices_to_replace] = "-"

    return sequences

# =============================================================================
def noisify_by_random_contigs(seqs, noise_percent, numgaps, mingapsize, mincontigsize):

    sequences = deepcopy(seqs)
    for seqind, seq in enumerate(sequences):

        amino_acid_length = len(seq)
        amino_acids_to_replace = int(amino_acid_length * noise_percent)

        # The idea is to build a gap queue, randomly putting each amino acid to replace into each gap
        # Then, we build a contig queue, randomly putting each amino acid into each contig
        gap_queue = np.zeros(numgaps).astype(int)
        for _ in range(amino_acids_to_replace):

            # Any gap can be considered
            valid_gaps = np.arange(numgaps)

            # ... so long as we have no underfilled gaps
            underfilled = gap_queue < mingapsize
            # If we have any underfilled, then consider only those until they are no longer underfilled
            if np.any(underfilled):
                valid_gaps = valid_gaps[np.where(underfilled)]

            # Once valid gap indices have been determined, randomly pick one to increment
            gap_queue[np.random.choice(valid_gaps)] += 1

        # There can always be one more contig than gaps (if there's a contig at beginnning and end of sequence)
        contig_queue = np.zeros(numgaps+1).astype(int)

        # We have to allocate all the amino acids NOT in gaps into contigs BETWEEN the gaps
        for _ in range(amino_acid_length - amino_acids_to_replace):

            # Any contig can be considered
            valid_contigs = np.arange(len(contig_queue))

            # ... so long as we have no underfilled contigs
            underfilled = contig_queue < mincontigsize

            # The exceptions are the first and last contigs. They are never considered underfilled
            underfilled[0] = False
            underfilled[-1] = False

            # If we have any underfilled, then consider only those until they are no longer underfilled
            if np.any(underfilled):
                valid_contigs = valid_contigs[np.where(underfilled)]

            # Once valid gap indices have been determined, randomly pick one to increment
            contig_queue[np.random.choice(valid_contigs)] += 1

        # Once we have determined gap_queue and contig_queue, we iterate over them to set the gaps equal
        # to our blank amino acid character

        sequence_pointer = 0
        iscontig=True
        while sequence_pointer < len(seq):
            if iscontig:
                # Don't do anything for the contig except increment the pointer and pop off the contig queue
                sequence_pointer += contig_queue[0]
                contig_queue = np.delete(contig_queue, 0)
            else:
                seq[sequence_pointer:sequence_pointer+gap_queue[0]] = '-'
                sequence_pointer += gap_queue[0]
                gap_queue = np.delete(gap_queue, 0)

            # We alternate between contigs and gaps
            iscontig = not iscontig

    return sequences

def get_sequences(fasta_file: str) -> List[np.array]:
    sequences = []
    lines = []
    with open(fasta_file, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append(np.array([c for c in "".join(parts)]))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append(np.array([c for c in "".join(parts)]))
    return sequences

class NoisificationMethod(Enum):
    RANDOMSCATTER=1
    RANDOMCONTIG=2

weights = None

In [9]:
ut.mkdir_if_not_exists(RESULTS_PATH)
ut.mkdir_if_not_exists(DATAPATH)

epochs = 20
noise_percent = 0.25
numgaps = 5
cwd = os.getcwd()

# change to the location of the data for reading
os.chdir(DATAPATH)
alltrainingdata = get_sequences("p5A_training_sequence.txt")
trainingdata = alltrainingdata[:1000]

In [12]:
noisemethod=NoisificationMethod.RANDOMSCATTER    
output_seqs = trainingdata
input_seqs = noisify_sequences(output_seqs, noise_percent, noisemethod, numgaps, mingapsize=3, mincontigsize=1)

In [14]:
len(output_seqs)

1000

In [15]:
len(input_seqs)

1000

In [18]:
percent_missings = np.array([0.20, 0.30, 0.40])
num_gaps = np.array([4, 6, 8, 10])
output_seqs_all = []
input_seqs_all = []

for n_gaps in num_gaps:
    for p_missing in percent_missings:
        output_seqs_temp = trainingdata
        input_seqs_temp = noisify_sequences(output_seqs_temp, p_missing, noisemethod, n_gaps, mingapsize=3, mincontigsize=1)
        output_seqs_all += output_seqs_temp
        input_seqs_all +=input_seqs_temp
print(len(output_seqs_all))  
print(len(input_seqs_all))
        

12000
12000


In [25]:
def introduce_gaps_and_errors(sequence, gap_probability=0.1, error_probability=0.05):
    new_sequence = []
    valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
    for aa in sequence:
        if np.random.rand() < gap_probability:
            new_sequence.append('-')
        else:
            if np.random.rand() < error_probability and aa in valid_amino_acids:
                new_sequence.append(random.choice(list(valid_amino_acids - {aa})))
            else:
                new_sequence.append(aa)
    return ''.join(new_sequence)

input_seq ="EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYDASTRATGIPDRFSGSGSGAD\
FLLTISSLEPEDFAMYYCQQYGRSPYTFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYP\
REAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQ\
GLSSPVTKSFNRGEC"
scaffold_seq = introduce_gaps_and_errors(input_seq, gap_probability=(0.06) , error_probability=0.20)
print(scaffold_seq)

RIVLTVK-GTL-LNRGELAA-SCRASQSKSLSWR-VYQQKPGTKPRLLIYDFSQ--TGWPDRFSQSGSGADFLL-IFSSEPEFFAK-YCQQYGRSPYTFGPGTKTDIKRTSAAPGVFIHPESDEQLQ-GTASVVCLL-NFY-REKKVQWK-ANALQSGN-QES-TQQDSKDFTQ-LSSTWTLSKADYEKMKVYACEV-H-GLSSPVTKSFARGEQ
